In [1]:
from datetime import datetime
#import geopy.distance # to compute distances between stations
import glob
import numpy as np
import os
import pandas as pd
import scipy.sparse as sp
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
#from torch_geometric_temporal.nn import STConv
from tqdm import tqdm
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import ChebConv
import pandas as pd
import numpy as np

/home/rxf131/ondemand/ubuntu2204/python310/torch_geometric/typing.py:47: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: libcudart.so.12: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/rxf131/ondemand/ubuntu2204/python310/torch_geometric/typing.py:101: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: libcudart.so.12: cannot open shared object file: No such file or directory
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [2]:
import os
import multiprocessing

# Get the total number of CPUs
total_cpus = os.cpu_count()
print(f"Total CPUs: {total_cpus}")

# Get the number of CPUs used by multiprocessing
used_cpus = multiprocessing.cpu_count()
print(f"Used CPUs: {used_cpus}")

import psutil

# Get the CPU usage
cpu_usage_percent = psutil.cpu_percent(interval=1)

print(f"Current CPU Usage: {cpu_usage_percent}%")

Total CPUs: 40
Used CPUs: 40
Current CPU Usage: 15.1%


## PV Data

In [3]:
df1 = pd.read_csv('rwb_simulated_PR_pw_36.1_-107.61.csv', sep=',')
df2 = pd.read_csv('rwb_simulated_PR_pw_39.86_-107.43.csv', sep=',')
df3 = pd.read_csv('rwb_simulated_PR_pw_38.49_-108.38.csv', sep=',')
df4 = pd.read_csv('rwb_simulated_PR_pw_39.74_-107.82.csv', sep=',')
df5 = pd.read_csv('rwb_simulated_PR_pw_39.17_-103.68.csv', sep=',')

In [4]:
x = pd.concat([df1.iloc[:,10:30],df2.iloc[:,10:30],df3.iloc[:,10:30],df4.iloc[:,10:30],df5.iloc[:,10:30]], axis=1, ignore_index=False)
x = x.fillna(0)
# x = x*100
np.max(x)

1.3344393247471684

In [5]:
location = pd.DataFrame(columns=['latd', 'lond'])
PLR = []

for i in range(10, 30):
  l = df1.columns[i].split('_')
  a = float(l[0])
  b = float(l[1])
  location.loc[i-10] = [a, b]
  PLR.append(float(l[2]))

for i in range(10, 30):
  l = df2.columns[i].split('_')
  a = float(l[0])
  b = float(l[1])
  location.loc[i+10] = [a, b]
  PLR.append(float(l[2]))

for i in range(10, 30):
  l = df3.columns[i].split('_')
  a = float(l[0])
  b = float(l[1])
  location.loc[i+30] = [a, b]
  PLR.append(float(l[2]))

for i in range(10, 30):
  l = df4.columns[i].split('_')
  a = float(l[0])
  b = float(l[1])
  location.loc[i+50] = [a, b]
  PLR.append(float(l[2]))

for i in range(10, 30):
  l = df5.columns[i].split('_')
  a = float(l[0])
  b = float(l[1])
  location.loc[i+70] = [a, b]
  PLR.append(float(l[2]))

PLR = [x*(-1) for x in PLR]
location

,latd,lond
0,36.0864,-107.6322
1,36.1014,-107.6733
2,36.0877,-107.6005
3,36.0979,-107.5536
4,36.1005,-107.5811
...,...,...
95,39.1736,-103.6255
96,39.1636,-103.6559
97,39.1593,-103.7698
98,39.1708,-103.7034


In [6]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

distance = np.zeros(shape=(100,100))
dist = []
for i in range(100):
    for j in range(100):
        d = haversine(location.iloc[i][1], location.iloc[i][0], location.iloc[j][1], location.iloc[j][0])
        distance[i][j] = d
        dist.append(d)

dist_std = np.std(dist)
distance = pd.DataFrame(distance)

# epsilon = 0, 0.25, 0.5, 0.75, 1
epsilon = 0.5
sigma = dist_std
W = np.zeros(shape=(100,100))

for i in range(100):
    for j in range(100):
        if i == j:
            W[i][j] = 0
        else:
            # Compute distance between stations
            d_ij = distance.loc[i][j]

            # Compute weight w_ij
            w_ij = np.exp(-d_ij**2 / sigma**2)

            if w_ij >= epsilon:
                W[i, j] = 1

W = pd.DataFrame(W)
W

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [7]:
data = torch.FloatTensor(120, 100)
N = 24*30
for i in range(120):
  for j in range(100):
    # avg = np.mean(x.iloc[i*N:(i+1)*N, j])
    # data[i][j] = avg
    M = x.iloc[i*N:(i+1)*N, j].to_numpy()
    data[i][j] = M[M!=0].mean()

data = data*100

In [18]:
import scipy.sparse as sp

x = data.T
A = W
G = sp.coo_matrix(W)
edge_index = torch.tensor(np.array([G.row, G.col]), dtype=torch.int64)
edge_weight = torch.tensor(G.data).float()
torch.max(x)

tensor(62.1814)

In [36]:
edge_index

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7,
         7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9,
         9, 9, 9, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3,
         4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7,
         8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1,
         2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
         6, 7, 8, 9]])

## Stock

In [11]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current directory
print("Current notebook file path:", current_directory)

Current notebook file path: /mnt/vstor/CSE_MSE_RXF131/cradle-members/sdle/yxf451/Git/21-pv-stgnn-bkp/topics/STGCN_DAE


In [12]:
import os
import pandas as pd

# Folder containing CSV files
folder_path = '/mnt/vstor/CSE_MSE_RXF131/cradle-members/sdle/yxf451/Git/21-pv-stgnn-bkp/topics/STGCN_DAE/Stock'

# List all files in the folder
all_files = os.listdir(folder_path)

# Filter only CSV files
csv_files = [file for file in all_files if file.endswith('.csv')]

# Specify the columns you want to read
columns_to_read = ['Adj Close']  # Replace with your actual column names

# Initialize an empty DataFrame to store the data
merged_df = pd.DataFrame()

# Loop through each CSV file and read the specified columns
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)

    # Read only the specified columns
    df = pd.read_csv(file_path, usecols=columns_to_read)

    # Rename columns with the CSV file name (without '.csv')
    df.columns = [csv_file.replace('.csv', '') + '_' + col for col in df.columns]

    # Concatenate the data to the merged DataFrame
    merged_df = pd.concat([merged_df, df], axis=1)

merged_df
# # Display the resulting DataFrame
def min_max_normalize(df):
    # Copy the original DataFrame to avoid modifying it in place
    normalized_df = df.copy()

    # Iterate through each column and apply min-max normalization
    for column in df.columns:
        min_val = df[column].min()
        max_val = df[column].max()

        # Min-max normalization formula: (x - min) / (max - min)
        normalized_df[column] = (df[column] - min_val) / (max_val - min_val)

    return normalized_df

# Apply min-max normalization
normalized_df = min_max_normalize(merged_df)
normalized_df = normalized_df * 100
normalized_df = normalized_df.iloc[:260,:]
normalized_df

,AAPL_Adj Close,ABBV_Adj Close,ABT_Adj Close,ACN_Adj Close,ADBE_Adj Close,AMD_Adj Close,AMGN_Adj Close,AMZN_Adj Close,AVGO_Adj Close,BAC_Adj Close,...,PG_Adj Close,QCOM_Adj Close,TMO_Adj Close,TSLA_Adj Close,UNH_Adj Close,V_Adj Close,VZ_Adj Close,WFC_Adj Close,WMT_Adj Close,XOM_Adj Close
0,2.544558,13.277270,4.345852,6.222199,4.420626,2.144714,14.235278,9.157560,4.864894,13.069831,...,6.583977,5.175687,3.997492,3.047108,14.164092,7.356254,64.797510,56.978983,5.086431,35.254216
1,0.364982,12.782527,0.000000,0.000000,0.000000,0.000000,6.776371,0.000000,3.956510,9.643498,...,0.000000,3.385837,0.000000,2.271788,7.169387,0.000000,57.102325,53.526864,0.000000,28.804969
2,1.177705,17.228049,4.756792,0.889368,2.988094,0.642692,13.392514,4.294397,4.996045,12.791998,...,0.243843,4.652308,1.459997,2.509750,9.240533,4.537370,58.349203,55.131357,5.388179,28.848190
3,0.000000,15.758181,1.045990,1.313691,3.626165,1.494801,16.354711,8.452150,3.112016,16.465279,...,1.764985,4.516602,1.308743,2.236684,7.504964,6.378115,62.232428,60.406742,6.799907,31.424435
4,0.595500,15.213228,2.565195,3.065266,5.994954,2.411901,19.574762,11.234945,4.716587,17.854332,...,0.928980,5.098151,5.307288,2.735731,9.615433,9.373591,68.146284,60.212293,8.308593,31.917208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,86.733697,79.967583,46.018557,67.565237,73.995449,68.833042,89.646544,59.548066,77.494065,35.801093,...,92.138714,55.129373,53.509815,52.558441,94.159080,86.437526,26.672618,59.928699,98.214941,89.157689
256,92.730057,77.404643,43.486232,69.822596,80.993385,73.411318,87.792535,63.783954,85.630605,33.241673,...,93.904655,58.550540,52.110902,51.214200,97.061874,87.594702,25.250619,58.909289,100.000000,84.699075
257,94.907150,77.142016,51.042603,72.786279,82.127735,74.877239,86.396583,65.167448,87.827814,41.196632,...,93.456583,62.387498,56.099258,56.188642,95.676630,90.989980,27.635905,64.734652,87.352922,86.037768
258,95.079579,77.477095,55.538507,75.047282,85.624621,76.097628,87.965019,66.499715,87.951755,40.331964,...,93.865120,61.132828,61.499823,56.479765,98.720198,94.321300,33.048659,64.622082,88.181276,86.664209


In [13]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Sample DataFrame
# Replace this with your actual DataFrame
data = {
    'Column1': [1, 2, 3, 4],
    'Column2': [5, 6, 7, 8],
    'Column3': [9, 10, 11, 12],
    # Add more columns as needed
}

df = pd.DataFrame(data)

# Function to compute cosine similarity between columns
def compute_cosine_similarity(df):
    # Compute cosine similarity matrix
    cosine_sim_matrix = cosine_similarity(df.T)  # Transpose to get columns as samples

    # Create a DataFrame from the similarity matrix
    adjacency_matrix = pd.DataFrame(cosine_sim_matrix, index=df.columns, columns=df.columns)

    return adjacency_matrix

# Call the function with your DataFrame
adjacency_matrix = compute_cosine_similarity(merged_df)

# Display the adjacency matrix
print("Adjacency Matrix:")
print(adjacency_matrix)


Adjacency Matrix:
                 AAPL_Adj Close  ABBV_Adj Close  ABT_Adj Close  ACN_Adj Close   
AAPL_Adj Close         1.000000        0.985586       0.968800       0.985598  \
ABBV_Adj Close         0.985586        1.000000       0.972777       0.982711   
ABT_Adj Close          0.968800        0.972777       1.000000       0.992942   
ACN_Adj Close          0.985598        0.982711       0.992942       1.000000   
ADBE_Adj Close         0.964204        0.948055       0.984848       0.985546   
AMD_Adj Close          0.987319        0.959075       0.964489       0.980956   
AMGN_Adj Close         0.962221        0.979087       0.989334       0.982898   
AMZN_Adj Close         0.953466        0.941215       0.987873       0.979431   
AVGO_Adj Close         0.983713        0.975749       0.942737       0.967104   
BAC_Adj Close          0.951830        0.961807       0.990337       0.986783   
BRK-B_Adj Close        0.977978        0.989679       0.987923       0.992313   
CMCSA_Adj 

In [14]:
# Function to compute correlation matrix
def compute_correlation_matrix(df):
    # Compute correlation matrix
    correlation_matrix = df.corr().abs()

    return correlation_matrix

# Call the function with your DataFrame
correlation_matrix = compute_correlation_matrix(normalized_df)
correlation_matrix[correlation_matrix<=0.75] = 0
correlation_matrix

,AAPL_Adj Close,ABBV_Adj Close,ABT_Adj Close,ACN_Adj Close,ADBE_Adj Close,AMD_Adj Close,AMGN_Adj Close,AMZN_Adj Close,AVGO_Adj Close,BAC_Adj Close,...,PG_Adj Close,QCOM_Adj Close,TMO_Adj Close,TSLA_Adj Close,UNH_Adj Close,V_Adj Close,VZ_Adj Close,WFC_Adj Close,WMT_Adj Close,XOM_Adj Close
AAPL_Adj Close,1.000000,0.888553,0.835096,0.930854,0.000000,0.913382,0.779497,0.000000,0.895973,0.000000,...,0.947701,0.843805,0.927840,0.901147,0.923524,0.878010,0.000000,0.0,0.918141,0.000000
ABBV_Adj Close,0.888553,1.000000,0.000000,0.785601,0.000000,0.000000,0.817756,0.000000,0.849173,0.000000,...,0.868976,0.000000,0.835733,0.000000,0.968426,0.000000,0.000000,0.0,0.789635,0.814668
ABT_Adj Close,0.835096,0.000000,1.000000,0.898797,0.776546,0.883380,0.000000,0.767864,0.000000,0.000000,...,0.822624,0.935783,0.921596,0.893397,0.000000,0.777723,0.000000,0.0,0.770920,0.000000
ACN_Adj Close,0.930854,0.785601,0.898797,1.000000,0.776719,0.933396,0.000000,0.000000,0.807020,0.753376,...,0.902944,0.886435,0.933729,0.923211,0.859254,0.846789,0.000000,0.0,0.835145,0.000000
ADBE_Adj Close,0.000000,0.000000,0.776546,0.776719,1.000000,0.841673,0.000000,0.893600,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
AMD_Adj Close,0.913382,0.000000,0.883380,0.933396,0.841673,1.000000,0.000000,0.000000,0.775973,0.000000,...,0.869876,0.894577,0.894111,0.910014,0.757026,0.809585,0.000000,0.0,0.835794,0.000000
AMGN_Adj Close,0.779497,0.817756,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.778392,0.000000,0.000000,0.000000,0.811358,0.757692,0.000000,0.0,0.807236,0.000000
AMZN_Adj Close,0.000000,0.000000,0.767864,0.000000,0.893600,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
AVGO_Adj Close,0.895973,0.849173,0.000000,0.807020,0.000000,0.775973,0.000000,0.000000,1.000000,0.000000,...,0.847971,0.000000,0.000000,0.000000,0.854421,0.806828,0.000000,0.0,0.846596,0.760602
BAC_Adj Close,0.000000,0.000000,0.000000,0.753376,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [9]:
# x = torch.tensor(merged_df.iloc[:260,:].values, dtype=torch.float32).T
x = torch.tensor(normalized_df.iloc[:260,:].values, dtype=torch.float32).T
A = correlation_matrix
G = sp.coo_matrix(A)
edge_index = torch.tensor(np.array([G.row, G.col]), dtype=torch.int64)
edge_weight = torch.tensor(G.data).float()
torch.max(x)

NameError: name 'normalized_df' is not defined

## GDP

In [3]:
df = pd.read_csv('/mnt/vstor/CSE_MSE_RXF131/cradle-members/sdle/yxf451/Git/21-pv-stgnn-bkp/topics/STGCN_DAE/GDP/GDP.csv', delimiter=',', encoding='utf-8', skiprows=4)

# Example: Filtering rows where 'Name' is in a list of names
G20_list = ['AUS', 'ARG', 'BRA', 'CAN', 'CHN', 'FRA', 'DEU', 'IND', 'IDN', 'ITA', 'JPN', 'MEX', 'RUS', 'SAU', 'ZAF', 'KOR', 'TUR', 'GBR', 'USA']
# filter = ['BRA', 'CAN', 'IND', 'ITA', 'JPN', 'MEX', 'RUS', 'SAU', 'GBR', 'USA']
# Applying the filter
condition = df['Country Code'].isin(G20_list)
filtered_df = df[condition]
filtered_df = filtered_df.reset_index(drop=True)
new_df = filtered_df.T.iloc[4:-1,:]
new_df = new_df.dropna(axis=1)
num_columns = len(new_df.columns)

# Create a dictionary to map current column names to new column names
new_column_names = {col: i for i, col in enumerate(new_df.columns)}

# Rename the columns
new_df.rename(columns=new_column_names, inplace=True)


# Display the resulting DataFrame
normalized_df = new_df.div(new_df.max())

# Display the resulting normalized DataFrame
normalized_df = normalized_df.iloc[3:,:]
normalized_df = normalized_df.astype('float32')
normalized_df = normalized_df*100
normalized_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
1963,1.285640,0.890150,2.104350,0.282281,2.865539,2.772218,1.430447,2.395973,1.108006,0.220242,1.199275,1.078785,2.507982,2.368880
1964,1.420608,0.801318,2.307554,0.332392,3.178218,3.023480,1.668502,2.622850,1.303321,0.191006,1.419897,1.161971,2.693352,2.609343
1965,1.550428,0.858722,2.547626,0.392113,3.432771,3.261022,1.759329,2.822245,1.450016,0.172333,1.544350,1.243864,2.920743,2.852250
1966,1.629912,1.081102,2.854746,0.427096,3.720430,3.477132,1.354926,3.057925,1.684024,0.216958,1.719716,1.465611,3.200760,3.101573
1967,1.817022,1.188246,3.068851,0.405727,4.022238,3.622661,1.481052,3.368399,1.973449,0.268136,1.878111,1.626147,3.384166,3.452949
1968,1.952592,1.296958,3.356795,0.394397,4.387786,3.451100,1.568214,3.651092,2.337254,0.337904,2.076104,1.819021,3.701493,3.737410
1969,2.189504,1.420849,3.698781,0.443717,4.797459,3.729878,1.726631,4.030675,2.745444,0.424015,2.296726,2.023444,4.005467,4.202753
1970,2.467061,1.617933,4.121442,0.515514,5.019013,4.185208,1.844042,4.707827,3.389619,0.497252,2.511690,1.810989,4.215197,4.630820
1971,2.698827,1.868001,4.654807,0.555585,5.611000,4.743236,1.989649,5.176015,3.828729,0.546866,2.771910,1.703036,4.574731,5.109364
1972,3.106212,2.233615,5.302445,0.632902,6.879731,5.443400,2.111117,6.030753,5.070358,0.599812,3.196177,2.133130,5.023466,5.350489


In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Sample DataFrame
# Replace this with your actual DataFrame
data = {
    'Column1': [1, 2, 3, 4],
    'Column2': [5, 6, 7, 8],
    'Column3': [9, 10, 11, 12],
    # Add more columns as needed
}

df = pd.DataFrame(data)

# Function to compute cosine similarity between columns
def compute_cosine_similarity(df):
    # Compute cosine similarity matrix
    cosine_sim_matrix = cosine_similarity(df.T)  # Transpose to get columns as samples

    # Create a DataFrame from the similarity matrix
    adjacency_matrix = pd.DataFrame(cosine_sim_matrix, index=df.columns, columns=df.columns)

    return adjacency_matrix

# Call the function with your DataFrame
adjacency_matrix = compute_cosine_similarity(normalized_df)
adjacency_matrix[adjacency_matrix<=0.95] = 0
# Display the adjacency matrix
print("Adjacency Matrix:")
print(adjacency_matrix)


Adjacency Matrix:
          0         1         2         3         4         5         6    
0   1.000000  0.984179  0.989144  0.000000  0.961498  0.961858  0.973649  \
1   0.984179  1.000000  0.975627  0.000000  0.957488  0.951461  0.000000   
2   0.989144  0.975627  1.000000  0.000000  0.988583  0.987821  0.950375   
3   0.000000  0.000000  0.000000  1.000000  0.000000  0.000000  0.985018   
4   0.961498  0.957488  0.988583  0.000000  1.000000  0.994629  0.000000   
5   0.961858  0.951461  0.987821  0.000000  0.994629  1.000000  0.000000   
6   0.973649  0.000000  0.950375  0.985018  0.000000  0.000000  1.000000   
7   0.000000  0.000000  0.979542  0.000000  0.997484  0.992106  0.000000   
8   0.000000  0.000000  0.000000  0.000000  0.973215  0.962139  0.000000   
9   0.986087  0.962403  0.983273  0.000000  0.964281  0.972227  0.973889   
10  0.979310  0.967123  0.991979  0.000000  0.984190  0.990922  0.000000   
11  0.991615  0.985221  0.984069  0.000000  0.959784  0.964889  0.9578

In [5]:
# Function to compute correlation matrix
def compute_correlation_matrix(df):
    # Compute correlation matrix
    correlation_matrix = df.corr().abs()

    return correlation_matrix

# Call the function with your DataFrame
correlation_matrix = compute_correlation_matrix(normalized_df)
# correlation_matrix[correlation_matrix<=0.25] = 0
correlation_matrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.000000,0.968333,0.984782,0.927076,0.935475,0.927517,0.958933,0.896876,0.776604,0.972980,0.959487,0.984369,0.959365,0.972863
1,0.968333,1.000000,0.956449,0.831699,0.929011,0.906225,0.883122,0.900823,0.802875,0.926449,0.934672,0.971666,0.902590,0.966803
2,0.984782,0.956449,1.000000,0.881727,0.974996,0.969861,0.932045,0.951096,0.841182,0.977102,0.981801,0.980761,0.977839,0.988502
3,0.927076,0.831699,0.881727,1.000000,0.789110,0.792243,0.989065,0.722481,0.579904,0.919240,0.847150,0.882636,0.903470,0.844475
4,0.935475,0.929011,0.974996,0.789110,1.000000,0.987235,0.857516,0.992659,0.920040,0.952006,0.970073,0.944741,0.958897,0.976343
5,0.927517,0.906225,0.969861,0.792243,0.987235,1.000000,0.858064,0.980502,0.897503,0.958656,0.981182,0.944755,0.967632,0.961796
6,0.958933,0.883122,0.932045,0.989065,0.857516,0.858064,1.000000,0.801812,0.664532,0.957035,0.900830,0.927603,0.945888,0.904565
7,0.896876,0.900823,0.951096,0.722481,0.992659,0.980502,0.801812,1.000000,0.937546,0.918226,0.949334,0.914010,0.931215,0.955790
8,0.776604,0.802875,0.841182,0.579904,0.920040,0.897503,0.664532,0.937546,1.000000,0.818689,0.861152,0.777888,0.845221,0.858978
9,0.972980,0.926449,0.977102,0.919240,0.952006,0.958656,0.957035,0.918226,0.818689,1.000000,0.972316,0.965537,0.986048,0.962995


In [14]:
x = torch.tensor(normalized_df.values, dtype=torch.float32).T
A = adjacency_matrix
G = sp.coo_matrix(A)
edge_index = torch.tensor(np.array([G.row, G.col]), dtype=torch.int64)
edge_weight = torch.tensor(G.data).float()
torch.max(x)

NameError: name 'normalized_df' is not defined

In [12]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss


m1 = [[1] for i in range(1,61)]
m2 = [[i] for i in range(1,61)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T

K = 12
d = [ [ 0 for y in range(60) ] for x in range(5) ]
for i in range(5):
  for j in range(60):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(5) ] for x in range(5) ]
for i in range(5):
  for j in range(5):
    W[i][j] = np.abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(5) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


class GAE1(torch.nn.Module):
    def __init__(self):
        super(GAE1, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(60, 30, 1)
        self.conv2 = GATConv(30, 15)
        self.conv3 = ChebConv(15, 30, 1)
        self.conv4 = GATConv(30, 60)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_1 = self.conv1(x, edge_index)
        h_2 = self.conv2(h_1, edge_index)
        h_3 = self.conv3(h_2, edge_index)
        h_a = self.conv4(h_3, edge_index)

        return h_a

class GAE2(torch.nn.Module):
    def __init__(self):
        super(GAE2, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(60, 30, 1)
        self.conv2 = GATConv(30, 15)
        self.conv3 = ChebConv(15, 30, 1)
        self.conv4 = GATConv(30, 60)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_5 = self.conv1(x, edge_index)
        h_6 = self.conv2(h_5, edge_index)
        h_7 = self.conv3(h_6, edge_index)
        h_f = self.conv4(h_7, edge_index)

        return h_f

# Dummy data (replace with your actual data)
# Assuming input features x, edge_index, and targets for regression
# x = torch.randn(100, 120)
# edge_index = torch.randint(0, 100, (2, 500))
target1 = x.clone()
target2 = x.clone()
target3 = x.clone()
target4 = x.clone()
target5 = x.clone()
target6 = x.clone()
target7 = x.clone()
target8 = x.clone()
# manager = multiprocessing.Manager()
# output_queue1 = manager.Queue()
# output_queue2 = manager.Queue()

# Number of epochs
num_epochs = 50

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, target):
    print("Called")
    output = model(x, edge_index)
    # output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])
    
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time aging: {elapsed_time} seconds")
    
# Function for backpropagation
def backpropagation_fluctuation1(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


def backpropagation_fluctuation2(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


def backpropagation_fluctuation3(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")

def backpropagation_fluctuation4(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


def backpropagation_fluctuation5(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


def backpropagation_fluctuation6(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


def backpropagation_fluctuation7(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")
    
start_time = time.time()

# Create instances of the models
model1 = GAE1()
model2 = GAE2()
model3 = GAE2()
model4 = GAE2()
model5 = GAE2()
model6 = GAE2()
model7 = GAE2()
model8 = GAE2()

# Optimizers for each model
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)
optimizer3 = torch.optim.Adam(model3.parameters(), lr=0.001)
optimizer4 = torch.optim.Adam(model4.parameters(), lr=0.001)
optimizer5 = torch.optim.Adam(model5.parameters(), lr=0.001)
optimizer6 = torch.optim.Adam(model6.parameters(), lr=0.001)
optimizer7 = torch.optim.Adam(model7.parameters(), lr=0.001)
optimizer8 = torch.optim.Adam(model8.parameters(), lr=0.001)
# start_time = time.time()

# Parallel forward propagation
# with ProcessPoolExecutor() as executor:
with ProcessPoolExecutor(max_workers=7) as executor:
    print("Submitting futures...")
    future1 = executor.submit(parallel_forward, model1, x, edge_index, target1)
    future2 = executor.submit(parallel_forward, model2, x, edge_index, target2)
    future3 = executor.submit(parallel_forward, model3, x, edge_index, target3)
    future4 = executor.submit(parallel_forward, model4, x, edge_index, target4)
    future5 = executor.submit(parallel_forward, model5, x, edge_index, target5)
    future6 = executor.submit(parallel_forward, model6, x, edge_index, target6)    
    future7 = executor.submit(parallel_forward, model7, x, edge_index, target7)   
    future8 = executor.submit(parallel_forward, model8, x, edge_index, target8) 
    print("Futures submitted.")

    print("Waiting for results...")
    output1, target1 = future1.result()
    output2, target2 = future2.result()
    output3, target3 = future3.result()
    output4, target4 = future4.result()
    output5, target5 = future5.result()
    output6, target6 = future6.result()
    output7, target7 = future7.result()
    output8, target8 = future8.result()
    print("Results received.")
    
    # Backpropagation
    print("Starting backpropagation...")
    executor.submit(backpropagation_aging, (model1, optimizer1, mse_loss, output2, target2))
    executor.submit(backpropagation_fluctuation1, (model2, optimizer2, mse_loss, output1, target1))
    executor.submit(backpropagation_fluctuation2, (model3, optimizer3, mse_loss, output1, target1))
    executor.submit(backpropagation_fluctuation3, (model4, optimizer4, mse_loss, output1, target1))
    executor.submit(backpropagation_fluctuation4, (model5, optimizer5, mse_loss, output1, target1))
    executor.submit(backpropagation_fluctuation5, (model6, optimizer6, mse_loss, output1, target1))
    executor.submit(backpropagation_fluctuation6, (model7, optimizer7, mse_loss, output1, target1))
    executor.submit(backpropagation_fluctuation7, (model8, optimizer8, mse_loss, output1, target1))
    print("Backpropagation submitted.")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

Submitting futures...


Called
Called
Called
Called
CalledCalled

Called
Futures submitted.
Waiting for results...
Called


RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x120 and 60x30)

In [18]:
# import torch
# from torch.nn import Linear
# from torch_geometric.nn import GATConv, GATv2Conv, GCNConv, TransformerConv, AGNNConv, FusedGATConv, ChebConv, SAGEConv, GraphConv, CuGraphGATConv, TAGConv, GMMConv
# from scipy.stats import linregress

# class GCN(torch.nn.Module):
#   def __init__(self):
#     super(GCN, self).__init__()
#     torch.manual_seed(42)

#     # self.conv1 = TransformerConv(260, 130)
#     # self.conv2 = GATConv(130, 65)
#     # self.conv3 = TransformerConv(65, 130)
#     # self.conv4 = GATConv(130, 260)

#     self.conv1 = nn.Linear(260, 130)
#     self.conv2 = nn.Linear(130, 65)
#     self.conv3 = nn.Linear(65, 130)
#     self.conv4 = nn.Linear(130, 260)


#     self.conv5 = nn.Linear(260, 130)
#     self.conv6 = nn.Linear(130, 65)
#     self.conv7 = nn.Linear(65, 130)
#     self.conv8 = nn.Linear(130, 260)


#   def forward(self, x, edge_index):


#     h_1 = self.conv1(x)
#     h_2 = self.conv2(h_1)
#     h_3 = self.conv3(h_2)
#     h_a = self.conv4(h_3)

#     h_5 = self.conv5(x)
#     h_6 = self.conv6(h_5)
#     h_7 = self.conv7(h_6)
#     h_f = self.conv8(h_7)
#     h_a = torch.nn.functional.avg_pool1d(h_a, kernel_size = 11, stride=1, padding = 11 // 2)

#     return h_a, h_f

# model = GCN()
# print(model)

In [9]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss

m1 = [[1] for i in range(1,261)]
m2 = [[i] for i in range(1,261)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T

K = 52
d = [ [ 0 for y in range(260) ] for x in range(5) ]
for i in range(5):
  for j in range(260):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(5) ] for x in range(5) ]
for i in range(5):
  for j in range(5):
    W[i][j] = np.abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(5) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


class GAE1(torch.nn.Module):
    def __init__(self):
        super(GAE1, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(260, 130, 1)
        self.conv2 = GATConv(130, 65)
        self.conv3 = ChebConv(65, 130, 1)
        self.conv4 = GATConv(130, 260)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_1 = self.conv1(x, edge_index)
        h_2 = self.conv2(h_1, edge_index)
        h_3 = self.conv3(h_2, edge_index)
        h_a = self.conv4(h_3, edge_index)

        return h_a

class GAE2(torch.nn.Module):
    def __init__(self):
        super(GAE2, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(260, 130, 1)
        self.conv2 = GATConv(130, 65)
        self.conv3 = ChebConv(65, 130, 1)
        self.conv4 = GATConv(130, 260)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_5 = self.conv1(x, edge_index)
        h_6 = self.conv2(h_5, edge_index)
        h_7 = self.conv3(h_6, edge_index)
        h_f = self.conv4(h_7, edge_index)

        return h_f

# Dummy data (replace with your actual data)
# Assuming input features x, edge_index, and targets for regression
# x = torch.randn(100, 120)
# edge_index = torch.randint(0, 100, (2, 500))
target1 = x.clone()
target2 = x.clone()
target3 = x.clone()
# target4 = x.clone()
# target5 = x.clone()
# target6 = x.clone()
# target7 = x.clone()
# target8 = x.clone()
# manager = multiprocessing.Manager()
# output_queue1 = manager.Queue()
# output_queue2 = manager.Queue()

# Number of epochs
num_epochs = 50

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, target):
    output = model(x, edge_index)
    # output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])
    
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time aging: {elapsed_time} seconds")
    
# Function for backpropagation
def backpropagation_fluctuation1(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


def backpropagation_fluctuation2(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation3(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")

# def backpropagation_fluctuation4(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation5(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation6(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation7(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")
    
start_time = time.time()

# Create instances of the models
model1 = GAE1()
model2 = GAE2()
model3 = GAE2()
# model4 = GAE2()
# model5 = GAE2()
# model6 = GAE2()
# model7 = GAE2()
# model8 = GAE2()

# Optimizers for each model
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)
optimizer3 = torch.optim.Adam(model3.parameters(), lr=0.001)
# optimizer4 = torch.optim.Adam(model4.parameters(), lr=0.001)
# optimizer5 = torch.optim.Adam(model5.parameters(), lr=0.001)
# optimizer6 = torch.optim.Adam(model6.parameters(), lr=0.001)
# optimizer7 = torch.optim.Adam(model7.parameters(), lr=0.001)
# optimizer8 = torch.optim.Adam(model8.parameters(), lr=0.001)
# start_time = time.time()

# Parallel forward propagation
# with ProcessPoolExecutor() as executor:
with ProcessPoolExecutor(max_workers=8) as executor:
    print("Submitting futures...")
    future1 = executor.submit(parallel_forward, model1, x, edge_index, target1)
    future2 = executor.submit(parallel_forward, model2, x, edge_index, target2)
    future3 = executor.submit(parallel_forward, model3, x, edge_index, target3)
    # future4 = executor.submit(parallel_forward, model4, x, edge_index, target4)
    # future5 = executor.submit(parallel_forward, model5, x, edge_index, target5)
    # future6 = executor.submit(parallel_forward, model6, x, edge_index, target6)    
    # future7 = executor.submit(parallel_forward, model7, x, edge_index, target7)   
    # future8 = executor.submit(parallel_forward, model8, x, edge_index, target8) 
    print("Futures submitted.")

    print("Waiting for results...")
    output1, target1 = future1.result()
    output2, target2 = future2.result()
    output3, target3 = future3.result()
    # output4, target4 = future4.result()
    # output5, target5 = future5.result()
    # output6, target6 = future6.result()
    # output7, target7 = future7.result()
    # output8, target8 = future8.result()
    print("Results received.")
    
    # Backpropagation
    print("Starting backpropagation...")
    executor.submit(backpropagation_aging, (model1, optimizer1, mse_loss, output2, target2))
    executor.submit(backpropagation_fluctuation1, (model2, optimizer2, mse_loss, output1, target1))
    executor.submit(backpropagation_fluctuation2, (model3, optimizer3, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation3, (model4, optimizer4, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation4, (model5, optimizer5, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation5, (model6, optimizer6, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation6, (model7, optimizer7, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation7, (model8, optimizer8, mse_loss, output1, target1))
    print("Backpropagation submitted.")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

Submitting futures...
Futures submitted.
Waiting for results...


RuntimeError: mat1 and mat2 shapes cannot be multiplied (14x60 and 260x130)

## Good code

In [19]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


class GAE1(torch.nn.Module):
    def __init__(self):
        super(GAE1, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_1 = self.conv1(x, edge_index)
        h_2 = self.conv2(h_1, edge_index)
        h_3 = self.conv3(h_2, edge_index)
        h_a = self.conv4(h_3, edge_index)

        return h_a

class GAE2(torch.nn.Module):
    def __init__(self):
        super(GAE2, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_5 = self.conv1(x, edge_index)
        h_6 = self.conv2(h_5, edge_index)
        h_7 = self.conv3(h_6, edge_index)
        h_f = self.conv4(h_7, edge_index)

        return h_f

# Dummy data (replace with your actual data)
# Assuming input features x, edge_index, and targets for regression
# x = torch.randn(100, 120)
# edge_index = torch.randint(0, 100, (2, 500))
target1 = x.clone()
target2 = x.clone()
# target3 = x.clone()
# target4 = x.clone()
# target5 = x.clone()
# target6 = x.clone()
# target7 = x.clone()
# target8 = x.clone()
# manager = multiprocessing.Manager()
# output_queue1 = manager.Queue()
# output_queue2 = manager.Queue()

# Number of epochs
num_epochs = 50

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, target):
    print("called")
    output = model(x, edge_index)
    # output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])
    
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target)
        # Backpropagation
        optimizer1.zero_grad()
        loss.backward()
        optimizer1.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time aging: {elapsed_time} seconds")
    
# Function for backpropagation
def backpropagation_fluctuation1(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer2.zero_grad()
        loss.backward()
        optimizer2.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation2(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation3(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")

# def backpropagation_fluctuation4(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation5(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation6(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation7(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")
    
start_time = time.time()

# Create instances of the models
model1 = GAE1()
model2 = GAE2()
# model3 = GAE2()
# model4 = GAE2()
# model5 = GAE2()
# model6 = GAE2()
# model7 = GAE2()
# model8 = GAE2()

# Optimizers for each model
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)
# optimizer3 = torch.optim.Adam(model3.parameters(), lr=0.001)
# optimizer4 = torch.optim.Adam(model4.parameters(), lr=0.001)
# optimizer5 = torch.optim.Adam(model5.parameters(), lr=0.001)
# optimizer6 = torch.optim.Adam(model6.parameters(), lr=0.001)
# optimizer7 = torch.optim.Adam(model7.parameters(), lr=0.001)
# optimizer8 = torch.optim.Adam(model8.parameters(), lr=0.001)
# start_time = time.time()

# Parallel forward propagation
# with ProcessPoolExecutor() as executor:
with ProcessPoolExecutor(max_workers=2) as executor:
    print("Submitting futures...")
    future1 = executor.submit(parallel_forward, model1, x, edge_index, target1)
    future2 = executor.submit(parallel_forward, model2, x, edge_index, target2)
    # future3 = executor.submit(parallel_forward, model3, x, edge_index, target3)
    # future4 = executor.submit(parallel_forward, model4, x, edge_index, target4)
    # future5 = executor.submit(parallel_forward, model5, x, edge_index, target5)
    # future6 = executor.submit(parallel_forward, model6, x, edge_index, target6)    
    # future7 = executor.submit(parallel_forward, model7, x, edge_index, target7)   
    # future8 = executor.submit(parallel_forward, model8, x, edge_index, target8) 
    print("Futures submitted.")

    print("Waiting for results...")
    output1, target1 = future1.result()
    output2, target2 = future2.result()
    # output3, target3 = future3.result()
    # output4, target4 = future4.result()
    # output5, target5 = future5.result()
    # output6, target6 = future6.result()
    # output7, target7 = future7.result()
    # output8, target8 = future8.result()
    print("Results received.")
    
    # Backpropagation
    print("Starting backpropagation...")
    executor.submit(backpropagation_aging, (model1, optimizer1, mse_loss, output2, target2))
    executor.submit(backpropagation_fluctuation1, (model2, optimizer2, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation2, (model3, optimizer3, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation3, (model4, optimizer4, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation4, (model5, optimizer5, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation5, (model6, optimizer6, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation6, (model7, optimizer7, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation7, (model8, optimizer8, mse_loss, output1, target1))
    print("Backpropagation submitted.")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

Submitting futures...
called
called


Futures submitted.
Waiting for results...
Results received.
Starting backpropagation...
Backpropagation submitted.
tensor([[ 15.7749, -58.2102, -34.8476,  ..., -27.2065, -48.2889, -17.7458],
        [ 15.7749, -58.2102, -34.8476,  ..., -27.2065, -48.2889, -17.7458],
        [ 15.7749, -58.2102, -34.8476,  ..., -27.2065, -48.2889, -17.7458],
        ...,
        [ 14.4217, -50.4337, -32.2782,  ..., -20.6976, -39.1249, -19.9011],
        [ 14.3259, -50.5901, -32.2198,  ..., -20.6795, -39.2114, -19.8774],
        [ 14.4347, -50.4026, -32.2870,  ..., -20.7025, -39.1107, -19.9015]])
Epoch 1/50, Model: GAE1, Loss: 7818.0390625
Epoch 1/50, Model: GAE2, Loss: 3197387.25
Epoch 2/50, Model: GAE1, Loss: 7818.0390625
tensor([[ 15.7749, -58.2102, -34.8476,  ..., -27.2065, -48.2889, -17.7458],
        [ 15.7749, -58.2102, -34.8476,  ..., -27.2065, -48.2889, -17.7458],
        [ 15.7749, -58.2102, -34.8476,  ..., -27.2065, -48.2889, -17.7458],
        ...,
        [ 14.4217, -50.4337, -32.2782,  ...,

In [ ]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


class GAE1(torch.nn.Module):
    def __init__(self):
        super(GAE1, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_1 = self.conv1(x, edge_index)
        h_2 = self.conv2(h_1, edge_index)
        h_3 = self.conv3(h_2, edge_index)
        h_a = self.conv4(h_3, edge_index)

        return h_a

class GAE2(torch.nn.Module):
    def __init__(self):
        super(GAE2, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_5 = self.conv1(x, edge_index)
        h_6 = self.conv2(h_5, edge_index)
        h_7 = self.conv3(h_6, edge_index)
        h_f = self.conv4(h_7, edge_index)

        return h_f

# Dummy data (replace with your actual data)
# Assuming input features x, edge_index, and targets for regression
# x = torch.randn(100, 120)
# edge_index = torch.randint(0, 100, (2, 500))
target1 = x.clone()
target2 = x.clone()
# target3 = x.clone()
# target4 = x.clone()
# target5 = x.clone()
# target6 = x.clone()
# target7 = x.clone()
# target8 = x.clone()
# manager = multiprocessing.Manager()
# output_queue1 = manager.Queue()
# output_queue2 = manager.Queue()

# Number of epochs
num_epochs = 50

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, target):
    print("called")
    output = model(x, edge_index)
    # output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])
    
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time aging: {elapsed_time} seconds")
    
# Function for backpropagation
def backpropagation_fluctuation1(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation2(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation3(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")

# def backpropagation_fluctuation4(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation5(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation6(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")


# def backpropagation_fluctuation7(args):
#     start_time = time.time()
#     model, optimizer, criterion, output, other_model_target = args
#     for epoch in range(num_epochs):
#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         # print(output)
#         # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
#         # loss = criterion(output, target) 
#         loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
#         torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time fluctuation: {elapsed_time} seconds")
    
start_time = time.time()

# Create instances of the models
model1 = GAE1()
model2 = GAE2()
# model3 = GAE2()
# model4 = GAE2()
# model5 = GAE2()
# model6 = GAE2()
# model7 = GAE2()
# model8 = GAE2()

# Optimizers for each model
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)
# optimizer3 = torch.optim.Adam(model3.parameters(), lr=0.001)
# optimizer4 = torch.optim.Adam(model4.parameters(), lr=0.001)
# optimizer5 = torch.optim.Adam(model5.parameters(), lr=0.001)
# optimizer6 = torch.optim.Adam(model6.parameters(), lr=0.001)
# optimizer7 = torch.optim.Adam(model7.parameters(), lr=0.001)
# optimizer8 = torch.optim.Adam(model8.parameters(), lr=0.001)
# start_time = time.time()

# Parallel forward propagation
# with ProcessPoolExecutor() as executor:
with ProcessPoolExecutor(max_workers=2) as executor:
    print("Submitting futures...")
    future1 = executor.submit(parallel_forward, model1, x, edge_index, target1)
    future2 = executor.submit(parallel_forward, model2, x, edge_index, target2)
    # future3 = executor.submit(parallel_forward, model3, x, edge_index, target3)
    # future4 = executor.submit(parallel_forward, model4, x, edge_index, target4)
    # future5 = executor.submit(parallel_forward, model5, x, edge_index, target5)
    # future6 = executor.submit(parallel_forward, model6, x, edge_index, target6)    
    # future7 = executor.submit(parallel_forward, model7, x, edge_index, target7)   
    # future8 = executor.submit(parallel_forward, model8, x, edge_index, target8) 
    print("Futures submitted.")

    print("Waiting for results...")
    output1, target1 = future1.result()
    output2, target2 = future2.result()
    # output3, target3 = future3.result()
    # output4, target4 = future4.result()
    # output5, target5 = future5.result()
    # output6, target6 = future6.result()
    # output7, target7 = future7.result()
    # output8, target8 = future8.result()
    print("Results received.")
    
    # Backpropagation
    print("Starting backpropagation...")
    executor.submit(backpropagation_aging, (model1, optimizer1, mse_loss, output2, target2))
    executor.submit(backpropagation_fluctuation1, (model2, optimizer2, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation2, (model3, optimizer3, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation3, (model4, optimizer4, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation4, (model5, optimizer5, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation5, (model6, optimizer6, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation6, (model7, optimizer7, mse_loss, output1, target1))
    # executor.submit(backpropagation_fluctuation7, (model8, optimizer8, mse_loss, output1, target1))
    print("Backpropagation submitted.")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

In [20]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


class GAE1(torch.nn.Module):
    def __init__(self):
        super(GAE1, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_1 = self.conv1(x, edge_index)
        h_2 = self.conv2(h_1, edge_index)
        h_3 = self.conv3(h_2, edge_index)
        h_a = self.conv4(h_3, edge_index)

        return h_a

class GAE2(torch.nn.Module):
    def __init__(self):
        super(GAE2, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_5 = self.conv1(x, edge_index)
        h_6 = self.conv2(h_5, edge_index)
        h_7 = self.conv3(h_6, edge_index)
        h_f = self.conv4(h_7, edge_index)

        return h_f


target1 = x.clone()
target2 = x.clone()

# Number of epochs
num_epochs = 50

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, target):
    output = model(x, edge_index)
    # output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])
    
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging(args):
    # start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    # Forward pass with the other model's output
    current_output = model(x, edge_index)
    loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss at each epoch
    print(f'Model: {model.__class__.__name__}, Loss: {loss.item()}')

    
# Function for backpropagation
def backpropagation_fluctuation1(args):
    model, optimizer, criterion, output, other_model_target = args
    current_output = model(x, edge_index)
    loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
    torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss at each epoch
    print(f'Model: {model.__class__.__name__}, Loss: {loss.item()}')




    
start_time = time.time()

# Create instances of the models
model1 = GAE1()
model2 = GAE2()


# Optimizers for each model
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)


for epoch in range(num_epochs):
    with ProcessPoolExecutor(max_workers=2) as executor:
        future1 = executor.submit(parallel_forward, model1, x, edge_index, target1)
        future2 = executor.submit(parallel_forward, model2, x, edge_index, target2)

    
        output1, target1 = future1.result()
        output2, target2 = future2.result()

        executor.submit(backpropagation_aging, (model1, optimizer1, mse_loss, output2, target2))
        executor.submit(backpropagation_fluctuation1, (model2, optimizer2, mse_loss, output1, target1))


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

Model: GAE1, Loss: 7830.443359375
Model: GAE2, Loss: 3221845.25
Model: GAE2, Loss: 1515561.5
Model: GAE1, Loss: 4551.16796875
Model: GAE2, Loss: 627302.125Model: GAE1, Loss: 2764.150390625

Model: GAE2, Loss: 324139.75
Model: GAE1, Loss: 1960.453369140625
Model: GAE2, Loss: 178554.65625
Model: GAE1, Loss: 1614.25732421875
Model: GAE2, Loss: 145713.640625
Model: GAE1, Loss: 1602.0245361328125
Model: GAE1, Loss: 1420.546142578125
Model: GAE2, Loss: 182462.515625
Model: GAE1, Loss: 1389.2672119140625
Model: GAE2, Loss: 160519.515625
Model: GAE2, Loss: 239274.375
Model: GAE1, Loss: 1203.0057373046875
Model: GAE2, Loss: 195066.828125
Model: GAE1, Loss: 1159.3184814453125
Model: GAE1, Loss: 998.6103515625Model: GAE2, Loss: 293449.09375

Model: GAE1, Loss: 939.1134643554688
Model: GAE2, Loss: 144955.5625
Model: GAE1, Loss: 808.5260009765625
Model: GAE2, Loss: 366854.0
Model: GAE2, Loss: 152465.015625
Model: GAE1, Loss: 759.7921752929688
Model: GAE2, Loss: 366912.125Model: GAE1, Loss: 651.8637

## Real Model Parallelism

In [11]:
import numpy as np
import os
import pandas as pd
import scipy.sparse as sp
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import math
from torch_geometric.nn import ChebConv


class TemporalConv(nn.Module):
    """
    Temporal Convolutional Layer.

    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        kernel_size (int): Convolutional kernel size.
        stride (int): Stride for convolution.
        padding (int): Padding for convolution.
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size, stride: int, padding: int):
        super(TemporalConv, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels, out_channels, (1, kernel_size), (1, stride), (0,padding))
        self.conv_2 = nn.Conv2d(in_channels, out_channels, (1, kernel_size), (1, stride), (0,padding))
        self.conv_3 = nn.Conv2d(in_channels, out_channels, (1, kernel_size), (1, stride), (0,padding))

    def forward(self, X: torch.FloatTensor) -> torch.FloatTensor:
        """
        Forward pass through temporal convolution block.

        Args:
            X (torch.FloatTensor): Input data of shape (batch_size, input_time_steps, num_nodes, in_channels).

        Returns:
            torch.FloatTensor: Output data of shape (batch_size, in_channels, num_nodes, input_time_steps).
        """
        X = X.permute(0, 3, 2, 1)
        P = self.conv_1(X)
        Q = torch.sigmoid(self.conv_2(X))
        PQ = P * Q
        H = F.relu(PQ + self.conv_3(X))
        H = H.permute(0, 3, 2, 1)
        return H



class TemporalDeConv1(nn.Module):
    """
    Temporal Deconvolutional Layer.

    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        kernel_size (int): Convolutional kernel size.
        stride (int): Stride for deconvolution.
        padding (int): Padding for deconvolution.
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size, stride: int, padding: int):
        super(TemporalDeConv1, self).__init__()
        self.conv_1 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size), (1, stride), (0,padding))
        self.conv_2 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride), (0,padding))
        self.conv_3 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride), (0,padding))

    def forward(self, X: torch.FloatTensor) -> torch.FloatTensor:
        """
        Forward pass through temporal convolution block.

        Args:
            X (torch.FloatTensor): Input data of shape (batch_size, input_time_steps, num_nodes, in_channels).

        Returns:
            torch.FloatTensor: Output data of shape (batch_size, in_channels, num_nodes, input_time_steps).
        """
        X = X.permute(0, 3, 2, 1)
        P = self.conv_1(X)
        Q = torch.sigmoid(self.conv_2(X))
        PQ = P * Q
        H = F.relu(PQ + self.conv_3(X))
        H = H.permute(0, 3, 2, 1)
        return H


class TemporalDeConv2(nn.Module):
    """
    Temporal Deconvolutional Layer (Simplified).

    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        kernel_size (int): Convolutional kernel size.
        stride (int): Stride for deconvolution.
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size, stride: int):
        super(TemporalDeConv2, self).__init__()
        self.conv_1 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride))
        self.conv_2 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride))
        self.conv_3 = nn.ConvTranspose2d(in_channels, out_channels, (1, kernel_size),(1, stride))

    def forward(self, X: torch.FloatTensor) -> torch.FloatTensor:
        """
        Forward pass through temporal convolution block.

        Args:
            X (torch.FloatTensor): Input data of shape (batch_size, input_time_steps, num_nodes, in_channels).

        Returns:
            torch.FloatTensor: Output data of shape (batch_size, in_channels, num_nodes, input_time_steps).
        """
        X = X.permute(0, 3, 2, 1)
        P = self.conv_1(X)
        Q = torch.sigmoid(self.conv_2(X))
        PQ = P * Q
        H = F.relu(PQ + self.conv_3(X))
        H = H.permute(0, 3, 2, 1)
        return H


class STConvEncoder(nn.Module):
    """
    Spatio-Temporal Convolutional Encoder Block.

    Args:
        num_nodes (int): Number of nodes in the input graph.
        in_channels (int): Number of input features.
        hidden_channels (int): Number of hidden features.
        out_channels (int): Number of output features.
        kernel_size (int): Length of the temporal kernel.
        stride (int): Stride for convolution.
        padding (int): Padding for convolution.
        K (int): Size of the Chebyshev filter for spatial convolution.
        normalization (str, optional): Type of normalization ("sym" for symmetric Laplacian or "rw" for random walk Laplacian).
        bias (bool, optional): Whether to include bias in convolutional layers.
    """

    def __init__(
        self,
        num_nodes: int,
        in_channels: int,
        hidden_channels: int,
        out_channels: int,
        kernel_size: int,
        stride: int,
        padding: int,
        K: int,
        normalization: str = "sym",
        bias: bool = True,
    ):
        super(STConvEncoder, self).__init__()
        self.num_nodes = num_nodes
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.K = K
        self.normalization = normalization
        self.bias = bias

        self._temporal_conv1 = TemporalConv(
            in_channels=in_channels,
            out_channels=hidden_channels,
            kernel_size=kernel_size, stride = stride, padding = padding,
        )

        self._graph_conv = ChebConv(
            in_channels=hidden_channels,
            out_channels=hidden_channels,
            K=K,
            normalization=normalization,
            bias=bias,
        )

        self._temporal_conv2 = TemporalConv(
            in_channels=hidden_channels,
            out_channels=out_channels,
            kernel_size=kernel_size, stride = stride, padding = padding,
        )
        

    def forward(self, X: torch.FloatTensor, edge_index: torch.LongTensor, edge_weight: torch.FloatTensor = None,) -> torch.FloatTensor:

        r"""Forward pass. If edge weights are not present the forward pass
        defaults to an unweighted graph.
        Arg types:
            * **X** (PyTorch FloatTensor) - Sequence of node features of shape (Batch size X Input time steps X Num nodes X In channels).
            * **edge_index** (PyTorch LongTensor) - Graph edge indices.
            * **edge_weight** (PyTorch LongTensor, optional)- Edge weight vector.
        Return types:
            * **T** (PyTorch FloatTensor) - Sequence of node features.
        """
        #print(X.shape)
        T_0 = self._temporal_conv1(X)
        #print(T_0.shape)
        T = torch.zeros_like(T_0).to(T_0.device)
        for b in range(T_0.size(0)):
            for t in range(T_0.size(1)):
                T[b][t] = self._graph_conv(T_0[b][t], edge_index, edge_weight)

        T = F.relu(T)
        T = self._temporal_conv2(T)
        return T

class STConvDecoder(nn.Module):
    """
    Spatio-Temporal Convolutional Decoder Block.

    Args:
        num_nodes (int): Number of nodes in the input graph.
        in_channels (int): Number of input features.
        hidden_channels (int): Number of hidden features.
        out_channels (int): Number of output features.
        kernel_size (int): Length of the temporal kernel for deconvolution.
        kernel_size_de (int): Length of the temporal kernel for deconvolution.
        stride (int): Stride for deconvolution.
        padding (int): Padding for deconvolution.
        K (int): Size of the Chebyshev filter for spatial convolution.
        normalization (str, optional): Type of normalization ("sym" for symmetric Laplacian or "rw" for random walk Laplacian).
        bias (bool, optional): Whether to include bias in convolutional layers.
    """

    def __init__(
        self,
        num_nodes: int,
        in_channels: int,
        hidden_channels: int,
        out_channels: int,
        kernel_size: int,
        kernel_size_de: int,
        stride: int,
        padding: int,
        K: int,
        normalization: str = "sym",
        bias: bool = True,
    ):
        super(STConvDecoder, self).__init__()
        self.num_nodes = num_nodes
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.K = K
        self.normalization = normalization
        self.bias = bias

        self._temporal_conv1 = TemporalDeConv1(
            in_channels=in_channels,
            out_channels=hidden_channels,
            kernel_size=kernel_size, stride = stride, padding = padding,
        )

        self._graph_conv = ChebConv(
            in_channels=hidden_channels,
            out_channels=hidden_channels,
            K=K,
            normalization=normalization,
            bias=bias,
        )

        self._temporal_conv2 = TemporalDeConv2(
            in_channels=hidden_channels,
            out_channels=out_channels,
            kernel_size=kernel_size_de, stride = stride,
        )

        self._batch_norm = nn.BatchNorm2d(num_nodes)

    def forward(self, X: torch.FloatTensor, edge_index: torch.LongTensor, edge_weight: torch.FloatTensor = None,) -> torch.FloatTensor:

        r"""Forward pass. If edge weights are not present the forward pass
        defaults to an unweighted graph.
        Arg types:
            * **X** (PyTorch FloatTensor) - Sequence of node features of shape (Batch size X Input time steps X Num nodes X In channels).
            * **edge_index** (PyTorch LongTensor) - Graph edge indices.
            * **edge_weight** (PyTorch LongTensor, optional)- Edge weight vector.
        Return types:
            * **T** (PyTorch FloatTensor) - Sequence of node features.
        """
        T_0 = self._temporal_conv1(X)
        T = torch.zeros_like(T_0).to(T_0.device)
        for b in range(T_0.size(0)):
            for t in range(T_0.size(1)):
                T[b][t] = self._graph_conv(T_0[b][t], edge_index, edge_weight)

        T = F.relu(T)
        T = self._temporal_conv2(T)
        return T


class STConvAE(torch.nn.Module):
    """
    Spatio-Temporal Convolutional Autoencoder.

    Args:
        device (str): The PyTorch device to use ('cpu' or 'cuda').
        num_nodes (int): Number of nodes in the input graph.
        channel_size_list (list): List of channel sizes for each layer.
        num_layers (int): Number of STConv blocks.
        kernel_size (int): Length of the temporal kernel.
        K (int): Size of the Chebyshev filter for spatial convolution.
        window_size (int): Number of historical time steps to consider.
        kernel_size_de (int): Length of the temporal kernel for deconvolution.
        stride (int): Stride for convolution and deconvolution.
        padding (int): Padding for convolution and deconvolution.
        normalization (str, optional): Type of normalization ("sym" for symmetric Laplacian or "rw" for random walk Laplacian).
        bias (bool, optional): Whether to include bias in convolutional layers.
    """

    def __init__(self, device, num_nodes, channel_size_list, num_layers,
                 kernel_size, K, window_size, kernel_size_de, stride, padding, \
                 normalization='sym', bias=True):

        super(STConvAE, self).__init__()
        self.layers = nn.ModuleList([])
        # add STConv blocks
        for l in range(num_layers):
            input_size, hidden_size, output_size = channel_size_list[l][0], channel_size_list[l][1], \
                                                   channel_size_list[l][2]
            if l == 0:
                self.layers.append(
                    STConvEncoder(num_nodes, input_size, hidden_size, output_size, kernel_size, stride, padding, K,
                                  normalization, bias))
            if l == 1:
                self.layers.append(
                    STConvDecoder(num_nodes, input_size, hidden_size, output_size, kernel_size, kernel_size_de, stride,
                                  padding, K, normalization, bias))


        for layer in self.layers:
            layer = layer.to(device)

    def forward(self, x, edge_index, edge_weight):
        for layer in self.layers:
            x = layer(x, edge_index, edge_weight)
        return x


In [12]:
# model parameters
num_nodes = 100
channels = np.array([[1, 4, 8], [8, 4, 1]])
kernel_size = 4 # size of temporal kernel
kernel_size_de = 2 # size of temporal deconv2
stride = 2
padding = 1
K = 3 # chebyshev filter size

# training parameters
learning_rate = 0.001
num_layers = 2 # number of STConv blocks
n_his = 60 # window size

In [15]:
# Add dimensions using unsqueeze
x = x.unsqueeze(0).unsqueeze(-1)
x.shape

torch.Size([1, 100, 120, 1])

In [13]:
device = torch.device("cuda") if torch.cuda.is_available() \
else torch.device("cpu")

In [26]:
edge_weight

tensor([1., 1., 2., 3., 4., 5., 6., 7., 8., 9., 1., 1., 1., 2., 3., 4., 5., 6.,
        7., 8., 2., 1., 1., 1., 2., 3., 4., 5., 6., 7., 3., 2., 1., 1., 1., 2.,
        3., 4., 5., 6., 4., 3., 2., 1., 1., 1., 2., 3., 4., 5., 5., 4., 3., 2.,
        1., 1., 1., 2., 3., 4., 6., 5., 4., 3., 2., 1., 1., 1., 2., 3., 7., 6.,
        5., 4., 3., 2., 1., 1., 1., 2., 8., 7., 6., 5., 4., 3., 2., 1., 1., 1.,
        9., 8., 7., 6., 5., 4., 3., 2., 1., 1.])

In [69]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


target1 = x.clone()
target2 = x.clone()

# Number of epochs
num_epochs = 10

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, edge_weight, target):
    print("called")
    output = model(x, edge_index, edge_weight)
    # output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])
    
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index, edge_weight)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        loss = criterion(current_output + output, x) 
        # loss = criterion(output, target)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time aging: {elapsed_time} seconds")
    
# Function for backpropagation
def backpropagation_fluctuation1(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index, edge_weight)
        print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        # loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        # torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        loss = criterion(current_output + output, x) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")



start_time = time.time()

# Create instances of the models
model1 = STConvAE(device, num_nodes, channels, num_layers, kernel_size, K, n_his, kernel_size_de, stride, padding, normalization = 'sym', bias = True)
model2 = STConvAE(device, num_nodes, channels, num_layers, kernel_size, K, n_his, kernel_size_de, stride, padding, normalization = 'sym', bias = True)


# Optimizers for each model
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)

# Parallel forward propagation
# with ProcessPoolExecutor() as executor:
with ProcessPoolExecutor(max_workers=2) as executor:
    print("Submitting futures...")
    future1 = executor.submit(parallel_forward, model1, x, edge_index, edge_weight, target1)
    future2 = executor.submit(parallel_forward, model2, x, edge_index, edge_weight, target2)
    print("Futures submitted.")

    print("Waiting for results...")
    output1, target1 = future1.result()
    output2, target2 = future2.result()

    print("Results received.")
    
    # Backpropagation
    print("Starting backpropagation...")
    executor.submit(backpropagation_aging, (model1, optimizer1, mse_loss, output2, target2))
    executor.submit(backpropagation_fluctuation1, (model2, optimizer2, mse_loss, output1, target1))

    print("Backpropagation submitted.")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

Submitting futures...
Futures submitted.
Waiting for results...
called
called
Results received.
Starting backpropagation...
Backpropagation submitted.
tensor([[[[ 0.0000],
          [ 0.0000],
          [ 0.0000],
          ...,
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],

         [[ 0.0000],
          [ 0.0000],
          [ 0.0000],
          ...,
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],

         [[11.8053],
          [11.0480],
          [11.0316],
          ...,
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],

         ...,

         [[ 1.7163],
          [ 1.8308],
          [ 1.8736],
          ...,
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],

         [[ 3.6383],
          [ 3.8312],
          [ 3.8488],
          ...,
          [ 0.0000],
          [ 0.0000],
          [ 0.0000]],

         [[ 3.5697],
          [ 3.7343],
          [ 3.7657],
          ...,
          [ 0.0000],
          [ 0.0000],

In [ ]:
model = STConvAE(device, num_nodes, channels, num_layers, kernel_size, K, n_his, kernel_size_de, stride, padding, normalization = 'sym', bias = True).to(device)
# define loss function
loss = nn.MSELoss()
# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 0.02) 

In [66]:
x[:,:,:60,:].shape

torch.Size([1, 100, 60, 1])

In [16]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss  

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


target1 = x.clone()
target2 = x.clone()

# Number of epochs
num_epochs = 10

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, edge_weight, target):
    print("called")
    output = model(x, edge_index, edge_weight)
    # output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])
    
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging1(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        print(x.shape)
        # Forward pass with the other model's output
        current_output = model(x[:,:,60:,:], edge_index, edge_weight)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        loss = criterion(current_output + output, x) 
        # loss = criterion(output, target)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time aging: {elapsed_time} seconds")

def backpropagation_aging2(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index, edge_weight)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        loss = criterion(current_output + output, x) 
        # loss = criterion(output, target)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time aging: {elapsed_time} seconds")
    
# Function for backpropagation
def backpropagation_fluctuation1(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index, edge_weight)
        print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        # loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        # torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        loss = criterion(current_output + output, x) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


def backpropagation_fluctuation2(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index, edge_weight)
        print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        # loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        # torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        loss = criterion(current_output + output, x) 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


start_time = time.time()

# Create instances of the models
model_a1 = STConvAE(device, num_nodes, channels, num_layers, kernel_size, K, n_his, kernel_size_de, stride, padding, normalization = 'sym', bias = True)
model_a2 = STConvAE(device, num_nodes, channels, num_layers, kernel_size, K, n_his, kernel_size_de, stride, padding, normalization = 'sym', bias = True)
model_f1 = STConvAE(device, num_nodes, channels, num_layers, kernel_size, K, n_his, kernel_size_de, stride, padding, normalization = 'sym', bias = True)
model_f2 = STConvAE(device, num_nodes, channels, num_layers, kernel_size, K, n_his, kernel_size_de, stride, padding, normalization = 'sym', bias = True)

# Optimizers for each model
optimizer1 = torch.optim.Adam(model_a1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model_a2.parameters(), lr=0.001)
optimizer3 = torch.optim.Adam(model_f1.parameters(), lr=0.001)
optimizer4 = torch.optim.Adam(model_f2.parameters(), lr=0.001)

# Parallel forward propagation
# with ProcessPoolExecutor() as executor:
with ProcessPoolExecutor(max_workers=4) as executor:
    print("Submitting futures...")
    future1 = executor.submit(parallel_forward, model_a1, x, edge_index, edge_weight, target1)
    future2 = executor.submit(parallel_forward, model_a2, x, edge_index, edge_weight, target2)
    future3 = executor.submit(parallel_forward, model_f1, x, edge_index, edge_weight, target2)
    future4 = executor.submit(parallel_forward, model_f2, x, edge_index, edge_weight, target2)
    print("Futures submitted.")

    print("Waiting for results...")
    output1, target1 = future1.result()
    output2, target2 = future2.result()
    output3, target3 = future3.result()
    output4, target4 = future4.result()

    print("Results received.")
    
    # Backpropagation
    print("Starting backpropagation...")
    executor.submit(backpropagation_aging1, (model_a1, optimizer1, mse_loss, output3, target2))
    executor.submit(backpropagation_aging2, (model_a2, optimizer2, mse_loss, output4, target2))
    executor.submit(backpropagation_fluctuation1, (model_f1, optimizer3, mse_loss, output1, target1))
    executor.submit(backpropagation_fluctuation2, (model_f2, optimizer4, mse_loss, output2, target1))

    print("Backpropagation submitted.")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

Submitting futures...
called
calledcalled
called



Futures submitted.
Waiting for results...
torch.Size([1, 100, 120, 1])
Results received.
Starting backpropagation...
Backpropagation submitted.
tensor([[[[3.5370],
          [3.5197],
          [3.5524],
          ...,
          [1.1176],
          [1.1224],
          [1.1220]],

         [[0.0000],
          [0.0000],
          [0.0000],
          ...,
          [0.5151],
          [0.5224],
          [0.5237]],

         [[1.4074],
          [1.3051],
          [1.4686],
          ...,
          [1.1063],
          [1.1159],
          [1.1230]],

         ...,

         [[1.0631],
          [0.6938],
          [0.8019],
          ...,
          [0.4624],
          [0.4597],
          [0.4624]],

         [[0.1867],
          [0.1696],
          [0.1746],
          ...,
          [0.9860],
          [0.9979],
          [0.9860]],

         [[1.0936],
          [1.1065],
          [1.1028],
          ...,
          [0.4570],
          [0.4590],
          [0.4570]]]])
tensor([[[[0.0000]

In [74]:
x[:,:,60:,:].shape

torch.Size([1, 100, 60, 1])

In [16]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


class GAE1(torch.nn.Module):
    def __init__(self):
        super(GAE1, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_1 = self.conv1(x, edge_index)
        h_2 = self.conv2(h_1, edge_index)
        h_3 = self.conv3(h_2, edge_index)
        h_a = self.conv4(h_3, edge_index)

        return h_a

class GAE2(torch.nn.Module):
    def __init__(self):
        super(GAE2, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_5 = self.conv1(x, edge_index)
        h_6 = self.conv2(h_5, edge_index)
        h_7 = self.conv3(h_6, edge_index)
        h_f = self.conv4(h_7, edge_index)

        return h_f


target1 = x.clone()
target2 = x.clone()

# Number of epochs
num_epochs = 50

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, target):
    output = model(x, edge_index)
    # output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])
    
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging(args):
    # start_time = time.time()
    model, optimizer, criterion, output, current_output, other_model_target = args
    loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss at each epoch
    print(f'Model: {model.__class__.__name__}, Loss: {loss.item()}')

    
# Function for backpropagation
def backpropagation_fluctuation1(args):
    model, optimizer, criterion, output, current_output, other_model_target = args
    loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
    torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss at each epoch
    print(f'Model: {model.__class__.__name__}, Loss: {loss.item()}')




    
start_time = time.time()

# Create instances of the models
model1 = GAE1()
model2 = GAE2()


# Optimizers for each model
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)



with ProcessPoolExecutor(max_workers=2) as executor:
    for epoch in range(num_epochs):
        future1 = executor.submit(parallel_forward, model1, x, edge_index, target1)
        future2 = executor.submit(parallel_forward, model2, x, edge_index, target2)

    
        output1, target1 = future1.result()
        output2, target2 = future2.result()

        executor.submit(backpropagation_aging, (model1, optimizer1, mse_loss, output2, output1, target2))
        executor.submit(backpropagation_fluctuation1, (model2, optimizer2, mse_loss, output1, output2, target1))


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

Training time: 9.984124422073364 seconds


In [10]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


class GAE1(torch.nn.Module):
    def __init__(self):
        super(GAE1, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_1 = self.conv1(x, edge_index)
        h_2 = self.conv2(h_1, edge_index)
        h_3 = self.conv3(h_2, edge_index)
        h_a = self.conv4(h_3, edge_index)

        return h_a

class GAE2(torch.nn.Module):
    def __init__(self):
        super(GAE2, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_5 = self.conv1(x, edge_index)
        h_6 = self.conv2(h_5, edge_index)
        h_7 = self.conv3(h_6, edge_index)
        h_f = self.conv4(h_7, edge_index)

        return h_f


target1 = x.clone()
target2 = x.clone()

# Number of epochs
num_epochs = 50

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, target):
    output = model(x, edge_index)
    # output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])
    
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging(args):
    # start_time = time.time()
    model, optimizer, criterion, output, current_output, other_model_target = args
    loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss at each epoch
    print(f'Model: {model.__class__.__name__}, Loss: {loss.item()}')

    
# Function for backpropagation
def backpropagation_fluctuation1(args):
    model, optimizer, criterion, output, current_output, other_model_target = args
    loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
    torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss at each epoch
    print(f'Model: {model.__class__.__name__}, Loss: {loss.item()}')




    
start_time = time.time()

# Create instances of the models
model1 = GAE1()
model2 = GAE2()


# Optimizers for each model
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)


for epoch in range(num_epochs):
    future1 = parallel_forward(model1, x, edge_index, target1)
    future2 = parallel_forward(model2, x, edge_index, target2)


    output1, target1 = future1
    output2, target2 = future2

    backpropagation_aging((args:=(model1, optimizer1, mse_loss, output2, output1, target2)))
    backpropagation_fluctuation1((args:=(model2, optimizer2, mse_loss, output1, output2, target1)))


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [12]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


class GAE1(torch.nn.Module):
    def __init__(self):
        super(GAE1, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_1 = self.conv1(x, edge_index)
        h_2 = self.conv2(h_1, edge_index)
        h_3 = self.conv3(h_2, edge_index)
        h_a = self.conv4(h_3, edge_index)

        return h_a

class GAE2(torch.nn.Module):
    def __init__(self):
        super(GAE2, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_5 = self.conv1(x, edge_index)
        h_6 = self.conv2(h_5, edge_index)
        h_7 = self.conv3(h_6, edge_index)
        h_f = self.conv4(h_7, edge_index)

        return h_f

# Dummy data (replace with your actual data)
# Assuming input features x, edge_index, and targets for regression
# x = torch.randn(100, 120)
# edge_index = torch.randint(0, 100, (2, 500))
target1 = x.clone()
target2 = x.clone()


# manager = multiprocessing.Manager()
# output_queue1 = manager.Queue()
# output_queue2 = manager.Queue()

# Number of epochs
num_epochs = 50

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, target):
    output = model(x, edge_index)
    # output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])
    
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time aging: {elapsed_time} seconds")
    
# Function for backpropagation
def backpropagation_fluctuation(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # print(output)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


start_time = time.time()

# Create instances of the models
model1 = GAE1()
model2 = GAE2()

# Optimizers for each model
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)

# start_time = time.time()

# Parallel forward propagation
with ProcessPoolExecutor() as executor:
    print("Submitting futures...")
    future1 = executor.submit(parallel_forward, model1, x, edge_index, target1)
    future2 = executor.submit(parallel_forward, model2, x, edge_index, target2)
    print("Futures submitted.")

    print("Waiting for results...")
    output1, target1 = future1.result()
    output2, target2 = future2.result()
    print("Results received.")
    
    # Backpropagation
    print("Starting backpropagation...")
    executor.submit(backpropagation_aging, (model1, optimizer1, mse_loss, output2, target2))
    executor.submit(backpropagation_fluctuation, (model2, optimizer2, mse_loss, output1, target1))
    print("Backpropagation submitted.")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

Submitting futures...
Futures submitted.
Waiting for results...


Process ForkProcess-40:
Process ForkProcess-23:
Process ForkProcess-15:
Process ForkProcess-31:
Process ForkProcess-28:
Process ForkProcess-24:
Process ForkProcess-16:
Process ForkProcess-25:
Process ForkProcess-26:
Process ForkProcess-5:
Traceback (most recent call last):
Process ForkProcess-33:
Process ForkProcess-34:
Process ForkProcess-17:
Process ForkProcess-41:
Process ForkProcess-18:
Process ForkProcess-8:
Process ForkProcess-32:
Process ForkProcess-6:
Process ForkProcess-35:
Process ForkProcess-10:
Process ForkProcess-9:
Process ForkProcess-36:
Process ForkProcess-39:
Process ForkProcess-11:
Process ForkProcess-7:
Process ForkProcess-27:
Traceback (most recent call last):
Process ForkProcess-19:
Process ForkProcess-20:
Process ForkProcess-38:
Process ForkProcess-22:
Traceback (most recent call last):
Process ForkProcess-30:
Traceback (most recent call last):
Process ForkProcess-21:
Process ForkProcess-42:
Traceback (most recent call last):
Traceback (most recent call last):
Pro

KeyboardInterrupt: 

In [13]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


class GAE1(torch.nn.Module):
    def __init__(self):
        super(GAE1, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_1 = self.conv1(x, edge_index)
        h_2 = self.conv2(h_1, edge_index)
        h_3 = self.conv3(h_2, edge_index)
        h_a = self.conv4(h_3, edge_index)

        return h_a

class GAE2(torch.nn.Module):
    def __init__(self):
        super(GAE2, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_5 = self.conv1(x, edge_index)
        h_6 = self.conv2(h_5, edge_index)
        h_7 = self.conv3(h_6, edge_index)
        h_f = self.conv4(h_7, edge_index)

        return h_f

# Dummy data (replace with your actual data)
# Assuming input features x, edge_index, and targets for regression
# x = torch.randn(100, 120)
# edge_index = torch.randint(0, 100, (2, 500))
target1 = x.clone()
target2 = x.clone()


# manager = multiprocessing.Manager()
# output_queue1 = manager.Queue()
# output_queue2 = manager.Queue()

# Number of epochs
num_epochs = 50

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, target):
    output = model(x, edge_index)
    # output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])
    
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging(args):
    # start_time = time.time()
    model, optimizer, criterion, output, current_output, other_model_target, epoch = args
    # print(output)
    # for epoch in range(num_epochs):
    # Forward pass with the other model's output
    current_output = model(x, edge_index)
    loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output)) + torch.var(current_output)
    # loss = criterion(output, target)
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss at each epoch
    print(f'Epoch {epoch+1}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

    # end_time = time.time()
    # elapsed_time = end_time - start_time
    # print(f"Training time aging: {elapsed_time} seconds")
    
# Function for backpropagation
def backpropagation_fluctuation(args):
    # start_time = time.time()
    model, optimizer, criterion, output, current_output, other_model_target, epoch = args
    # for epoch in range(num_epochs):
        # Forward pass with the other model's output
    current_output = model(x, edge_index)
    # print(output)
    # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
    # loss = criterion(output, target) 
    loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
    torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss at each epoch
    print(f'Epoch {epoch+1}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    # end_time = time.time()
    # elapsed_time = end_time - start_time
    # print(f"Training time fluctuation: {elapsed_time} seconds")


start_time = time.time()

# Create instances of the models
model1 = GAE1()
model2 = GAE2()

# Optimizers for each model
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)

# start_time = time.time()

# # Parallel forward propagation

with ProcessPoolExecutor() as executor:
    for epoch in range(num_epochs):
        # print("Submitting futures...")
        future1 = executor.submit(parallel_forward, model1, x, edge_index, target1)
        future2 = executor.submit(parallel_forward, model2, x, edge_index, target2)
        # print("Futures submitted.")
    
        # print("Waiting for results...")
        output1, target1 = future1.result()
        output2, target2 = future2.result()
        # print("Results received.")
        
        # Backpropagation
        # print("Starting backpropagation...")
        executor.submit(backpropagation_aging, (model1, optimizer1, mse_loss, output2, output1, target2, epoch))
        executor.submit(backpropagation_fluctuation, (model2, optimizer2, mse_loss, output1, output2, target1, epoch))
        # print("Backpropagation submitted.")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")








# Alternate between forward and backward propagation for each epoch

# for epoch in range(num_epochs):
#     # Forward propagation
#     process1 = multiprocessing.Process(target=parallel_forward, args=(model1, input_data, output_queue1, target1))
#     process2 = multiprocessing.Process(target=parallel_forward, args=(model2, input_data, output_queue2, target2))

#     process1.start()
#     process2.start()

#     process1.join()
#     process2.join()

#     # Backward propagation
#     backprop_process1 = multiprocessing.Process(target=backpropagation, args=(model1, optimizer1, nn.CrossEntropyLoss(), output_queue2, target2))
#     backprop_process2 = multiprocessing.Process(target=backpropagation, args=(model2, optimizer2, nn.CrossEntropyLoss(), output_queue1, target1))

#     backprop_process1.start()
#     backprop_process2.start()

#     backprop_process1.join()
#     backprop_process2.join()

Process ForkProcess-76:
Process ForkProcess-65:
Process ForkProcess-45:
Process ForkProcess-55:
Process ForkProcess-72:
Process ForkProcess-60:
Process ForkProcess-74:
Process ForkProcess-82:
Process ForkProcess-63:
Process ForkProcess-64:
Process ForkProcess-61:
Process ForkProcess-80:
Process ForkProcess-57:
Process ForkProcess-47:
Process ForkProcess-49:
Process ForkProcess-56:
Process ForkProcess-69:
Process ForkProcess-77:
Process ForkProcess-62:
Process ForkProcess-81:
Process ForkProcess-78:
Process ForkProcess-53:
Traceback (most recent call last):
Process ForkProcess-73:
Process ForkProcess-52:
Process ForkProcess-67:
Process ForkProcess-51:
Traceback (most recent call last):
Process ForkProcess-71:
Process ForkProcess-70:
Process ForkProcess-68:
Process ForkProcess-66:
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Process ForkProcess-59:
Process ForkProcess-46:
Process ForkProcess-75:
Process ForkProcess-79:
Process ForkProces

KeyboardInterrupt: 

In [35]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


class GAE1(torch.nn.Module):
    def __init__(self):
        super(GAE1, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_1 = self.conv1(x, edge_index)
        h_2 = self.conv2(h_1, edge_index)
        h_3 = self.conv3(h_2, edge_index)
        h_a = self.conv4(h_3, edge_index)

        return h_a

class GAE2(torch.nn.Module):
    def __init__(self):
        super(GAE2, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_5 = self.conv1(x, edge_index)
        h_6 = self.conv2(h_5, edge_index)
        h_7 = self.conv3(h_6, edge_index)
        h_f = self.conv4(h_7, edge_index)

        return h_f

# Dummy data (replace with your actual data)
# Assuming input features x, edge_index, and targets for regression
# x = torch.randn(100, 120)
# edge_index = torch.randint(0, 100, (2, 500))
target1 = x.clone()
target2 = x.clone()

# Number of epochs
num_epochs = 100

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, target):
    output = model(x, edge_index)
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output)) + torch.var(h_a)
        # loss = criterion(output, target)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time aging: {elapsed_time} seconds")
    
# Function for backpropagation
def backpropagation_fluctuation(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


start_time = time.time()

# Create instances of the models
model1 = GAE1()
model2 = GAE2()

# Optimizers for each model
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)

# start_time = time.time()

# Parallel forward propagation
with ProcessPoolExecutor() as executor:
    print("Submitting futures...")
    future1 = executor.submit(parallel_forward, model1, x, edge_index, target1)
    future2 = executor.submit(parallel_forward, model2, x, edge_index, target2)
    print("Futures submitted.")

    print("Waiting for results...")
    output1, target1 = future1.result()
    output2, target2 = future2.result()
    print("Results received.")
    
    # Backpropagation
    print("Starting backpropagation...")
    executor.submit(backpropagation_aging, (model1, optimizer1, mse_loss, output2, target2))
    executor.submit(backpropagation_fluctuation, (model2, optimizer2, mse_loss, output1, target1))
    print("Backpropagation submitted.")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

Submitting futures...
Futures submitted.
Waiting for results...
Results received.
Starting backpropagation...
Backpropagation submitted.
Epoch 1/100, Model: GAE2, Loss: 3221845.25
Epoch 2/100, Model: GAE2, Loss: 1516573.125
Epoch 3/100, Model: GAE2, Loss: 655325.5625
Epoch 4/100, Model: GAE2, Loss: 267109.09375
Epoch 5/100, Model: GAE2, Loss: 230921.734375
Epoch 6/100, Model: GAE2, Loss: 386658.0625
Epoch 7/100, Model: GAE2, Loss: 678982.625
Epoch 8/100, Model: GAE2, Loss: 1020604.9375
Epoch 9/100, Model: GAE2, Loss: 1501049.5
Epoch 10/100, Model: GAE2, Loss: 2017814.0
Epoch 11/100, Model: GAE2, Loss: 2556926.75
Epoch 12/100, Model: GAE2, Loss: 3122349.0
Epoch 13/100, Model: GAE2, Loss: 3723630.5
Epoch 14/100, Model: GAE2, Loss: 4436397.0
Epoch 15/100, Model: GAE2, Loss: 4813869.0
Epoch 16/100, Model: GAE2, Loss: 5121729.0
Epoch 17/100, Model: GAE2, Loss: 5276498.0
Epoch 18/100, Model: GAE2, Loss: 5340437.5
Epoch 19/100, Model: GAE2, Loss: 5230909.0
Epoch 20/100, Model: GAE2, Loss: 503

In [20]:
(122.00995969772339+123.03065609931946)/127.30234575271606

1.924871174589607

In [43]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GATConv, GATv2Conv, GCNConv, TransformerConv, AGNNConv, FusedGATConv, ChebConv, SAGEConv, GraphConv, CuGraphGATConv, TAGConv, GMMConv
from scipy.stats import linregress

class GAE1(torch.nn.Module):
  def __init__(self):
    super(GAE1, self).__init__()
    torch.manual_seed(42)

    self.conv1 = ChebConv(120, 60, 1)
    self.conv2 = GATConv(60, 30)
    self.conv3 = ChebConv(30, 60, 1)
    self.conv4 = GATConv(60, 120)


  def forward(self, x, edge_index):
    h_1 = self.conv1(x, edge_index)
    h_2 = self.conv2(h_1, edge_index)
    h_3 = self.conv3(h_2, edge_index)
    h_a = self.conv4(h_3, edge_index)

    return h_a


class GAE2(torch.nn.Module):
  def __init__(self):
    super(GAE2, self).__init__()
    torch.manual_seed(42)

    self.conv1 = ChebConv(120, 60, 1)
    self.conv2 = GATConv(60, 30)
    self.conv3 = ChebConv(30, 60, 1)
    self.conv4 = GATConv(60, 120)


  def forward(self, x, edge_index):
    h_5 = self.conv1(x, edge_index)
    h_6 = self.conv2(h_5, edge_index)
    h_7 = self.conv3(h_6, edge_index)
    h_f = self.conv4(h_7, edge_index)

    return h_f

model1 = GAE1()
model2 = GAE2()

print(model1)
print(model2)

GAE1(
  (conv1): ChebConv(120, 60, K=1, normalization=sym)
  (conv2): GATConv(60, 30, heads=1)
  (conv3): ChebConv(30, 60, K=1, normalization=sym)
  (conv4): GATConv(60, 120, heads=1)
)
GAE2(
  (conv1): ChebConv(120, 60, K=1, normalization=sym)
  (conv2): GATConv(60, 30, heads=1)
  (conv3): ChebConv(30, 60, K=1, normalization=sym)
  (conv4): GATConv(60, 120, heads=1)
)


In [65]:
def train_GAE1(model, x, edge_index, epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    for epoch in range(epochs):
        h_a = model(x, edge_index)
        loss = torch.var(h_a)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print(f"Model {model.__class__.__name__}, Epoch {epoch+1}, Loss: {loss.item()}")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time for {model.__class__.__name__}: {elapsed_time} seconds")
    

def train_GAE2(model, x, edge_index, epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    start_time = time.time()
    for epoch in range(epochs):
        h_f = model(x, edge_index)
        loss = torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(h_f, 0, 1))[1])) + torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(h_f, E.T).long(), L.long()), torch.matmul(E, h_f.T).long()), 0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print(f"Model {model.__class__.__name__}, Epoch {epoch+1}, Loss: {loss.item()}")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time for {model.__class__.__name__}: {elapsed_time} seconds")
    
# def train_GAE2(x, edge_index, Lambda1, Lambda2):
#   optimizer.zero_grad()
#   h_a, h_f = model(x, edge_index)
#   loss = criterion(h_a+h_f, x)  + \
#   10*torch.std(torch.diff(h_a)) +\
#   Lambda1 * torch.var(h_a) +\
#   Lambda2 * torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(h_f, 0, 1))[1])) +\
#   Lambda3 * torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(h_f, E.T).long(), L.long()), torch.matmul(E, h_f.T).long()), 0))
#   loss.backward()
#   optimizer.step()
#   return loss, criterion(h_a+h_f, x), h_a, h_f



# def train_model(model, data, epochs):
#     criterion = nn.MSELoss()
#     optimizer = optim.SGD(model.parameters(), lr=0.01)
#     start_time = time.time()

#     for epoch in range(epochs):
#         inputs, targets = data
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         print(f"Model {model.__class__.__name__}, Epoch {epoch+1}, Loss: {loss.item()}")

#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time for {model.__class__.__name__}: {elapsed_time} seconds")


In [68]:
import torch
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from scipy.stats import linregress
from concurrent.futures import ThreadPoolExecutor, wait  # Import wait from concurrent.futures

# Number of epochs for training
num_epochs = 500

start_time = time.time()

if __name__ == '__main__':
    # Create two instances of the GAE models
    model1 = GAE1()
    model2 = GAE2()

    # Create a ThreadPoolExecutor
    with ThreadPoolExecutor() as executor:
        # Submit tasks to the ThreadPoolExecutor
        future1 = executor.submit(train_GAE1, model1, x, edge_index, num_epochs)
        future2 = executor.submit(train_GAE2, model2, x, edge_index, num_epochs)

        # Wait for both tasks to finish
        wait([future1, future2])

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

Training time for GAE1: 10.309388399124146 seconds
Training time for GAE2: 11.01285719871521 seconds
Training time: 11.029053688049316 seconds


## Use this code

In [41]:
import torch
import time
import torch.nn as nn
import torch.optim as optim
from concurrent.futures import ProcessPoolExecutor
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from torch.nn.functional import mse_loss

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W


class GAE1(torch.nn.Module):
    def __init__(self):
        super(GAE1, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_1 = self.conv1(x, edge_index)
        h_2 = self.conv2(h_1, edge_index)
        h_3 = self.conv3(h_2, edge_index)
        h_a = self.conv4(h_3, edge_index)

        return h_a

class GAE2(torch.nn.Module):
    def __init__(self):
        super(GAE2, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120)  # Adjust output size for regression

    def forward(self, x, edge_index):
        h_5 = self.conv1(x, edge_index)
        h_6 = self.conv2(h_5, edge_index)
        h_7 = self.conv3(h_6, edge_index)
        h_f = self.conv4(h_7, edge_index)

        return h_f

# Dummy data (replace with your actual data)
# Assuming input features x, edge_index, and targets for regression
# x = torch.randn(100, 120)
# edge_index = torch.randint(0, 100, (2, 500))
target1 = x.clone()
target2 = x.clone()


# manager = multiprocessing.Manager()
# output_queue1 = manager.Queue()
# output_queue2 = manager.Queue()

# Number of epochs
num_epochs = 100

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, target):
    output = model(x, edge_index)
    # output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])
    
    return output.detach(), target.detach() 

# Function for backpropagation
def backpropagation_aging(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time aging: {elapsed_time} seconds")
    
# Function for backpropagation
def backpropagation_fluctuation(args):
    start_time = time.time()
    model, optimizer, criterion, output, other_model_target = args
    for epoch in range(num_epochs):
        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        # loss = criterion(output, target) 
        loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training time fluctuation: {elapsed_time} seconds")


start_time = time.time()

# Create instances of the models
model1 = GAE1()
model2 = GAE2()

# Optimizers for each model
optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001)

# start_time = time.time()

# Parallel forward propagation
with ProcessPoolExecutor() as executor:
    print("Submitting futures...")
    future1 = executor.submit(parallel_forward, model1, x, edge_index, target1)
    future2 = executor.submit(parallel_forward, model2, x, edge_index, target2)
    print("Futures submitted.")

    print("Waiting for results...")
    output1, target1 = future1.result()
    output2, target2 = future2.result()
    print("Results received.")
    
    # Backpropagation
    print("Starting backpropagation...")
    executor.submit(backpropagation_aging, (model1, optimizer1, mse_loss, output2, target2))
    executor.submit(backpropagation_fluctuation, (model2, optimizer2, mse_loss, output1, target1))
    print("Backpropagation submitted.")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

False

## WORKing

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import multiprocessing
import concurrent
from concurrent.futures import ProcessPoolExecutor


class GAE1(torch.nn.Module):
  def __init__(self):
    super(GAE1, self).__init__()
    torch.manual_seed(42)
    self.conv1 = ChebConv(120, 60, 1)
    self.conv2 = GATConv(60, 30)
    self.conv3 = ChebConv(30, 60, 1)
    self.conv4 = GATConv(60, 120)


  def forward(self, x, edge_index):
    h_1 = self.conv1(x, edge_index)
    h_2 = self.conv2(h_1, edge_index)
    h_3 = self.conv3(h_2, edge_index)
    h_a = self.conv4(h_3, edge_index)

    return h_a


class GAE2(torch.nn.Module):
  def __init__(self):
    super(GAE2, self).__init__()
    torch.manual_seed(42)
    self.conv1 = ChebConv(120, 60, 1)
    self.conv2 = GATConv(60, 30)
    self.conv3 = ChebConv(30, 60, 1)
    self.conv4 = GATConv(60, 120)


  def forward(self, x, edge_index):
    h_5 = self.conv1(x, edge_index)
    h_6 = self.conv2(h_5, edge_index)
    h_7 = self.conv3(h_6, edge_index)
    h_f = self.conv4(h_7, edge_index)

    return h_f




manager = multiprocessing.Manager()
output_queue1 = manager.Queue()
output_queue2 = manager.Queue()

# # Function for parallel forward propagation
# def parallel_forward(model, x, edge_index, output_queue):
#     with torch.no_grad():
#         output = model(x, edge_index)

#     # Put the output into the queue
#     output_queue.put(output)
    

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, output_queue):
    output = model(x, edge_index)
    output_queue.put(output.detach())
    
    return output.detach()
    
        
def backpropagation_aging(model, optimizer, criterion, output_queue):
    for epoch in range(num_epochs):
        # Get the latest output from the other model
        output = output_queue.get()

        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # loss = criterion(current_output + output, x) + torch.var(torch.diff(current_output))
        loss = criterion(current_output + output, x)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch + 1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')



def backpropagation_fluctuation(model, optimizer, criterion, output_queue):
    for epoch in range(num_epochs):
        # Get the latest output from the other model
        other_model_output = output_queue.get()

        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        # loss = criterion(current_output + output, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(current_output, 0, 1))[1])) +\
        # torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(current_output, E.T).long(), L.long()), torch.matmul(E, current_output.T).long()), 0))
        loss = criterion(current_output + output, x) 

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch + 1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')




# ... (your model instances, optimizers, etc.)
criterion = nn.MSELoss() 
num_epochs = 10
model1 = GAE1()
model2 = GAE2()

optimizer1 = optim.SGD(model1.parameters(), lr=0.01)
optimizer2 = optim.SGD(model2.parameters(), lr=0.01)


with ProcessPoolExecutor() as executor:
    print("Submitting futures...")
    future1 = executor.submit(parallel_forward, model1, x, edge_index, output_queue1)
    future2 = executor.submit(parallel_forward, model2, x, edge_index, output_queue2)
    print("Futures submitted.")

    print("Waiting for results...")
    output1 = future1.result()
    output2 = future2.result()
    print("Results received.")

    # Backpropagation
    print("Starting backpropagation...")
    executor.submit(backpropagation_aging, model1, optimizer1, criterion, output_queue2)
    executor.submit(backpropagation_fluctuation, model2, optimizer2, criterion, output_queue1)
    print("Backpropagation submitted.")

Submitting futures...
Futures submitted.
Waiting for results...
Results received.
Starting backpropagation...
Backpropagation submitted.
Epoch 1/10, Model: GAE1, Loss: 5935.828125


Process ForkProcess-508:
Process ForkProcess-521:
Process ForkProcess-529:
Process ForkProcess-525:
Traceback (most recent call last):
Process ForkProcess-509:
Process ForkProcess-523:
Process ForkProcess-524:
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 102, in get
    with self._rlock:
Traceback (most recent call last):
Process ForkProcess-507:
Process ForkProcess-516:
Process ForkProcess-522:
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkProcess-511:
Traceback (most recent call last):
Process ForkProcess-518:
Process ForkProcess-517:
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process ForkProcess-520:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Process ForkProcess-515:
Process 

KeyboardInterrupt: 

  File "/usr/lib/python3.10/concurrent/futures/process.py", line 240, in _process_worker
    call_item = call_queue.get(block=True)
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 102, in get
    with self._rlock:
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.10/multiproces

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import multiprocessing
from concurrent.futures import ProcessPoolExecutor

class GAE1(torch.nn.Module):
  def __init__(self):
    super(GAE1, self).__init__()
    torch.manual_seed(42)

    self.conv1 = ChebConv(120, 60, 1)
    self.conv2 = GATConv(60, 30)
    self.conv3 = ChebConv(30, 60, 1)
    self.conv4 = GATConv(60, 120)


  def forward(self, x, edge_index):
    h_1 = self.conv1(x, edge_index)
    h_2 = self.conv2(h_1, edge_index)
    h_3 = self.conv3(h_2, edge_index)
    h_a = self.conv4(h_3, edge_index)

    return h_a


class GAE2(torch.nn.Module):
  def __init__(self):
    super(GAE2, self).__init__()
    torch.manual_seed(42)

    self.conv1 = ChebConv(120, 60, 1)
    self.conv2 = GATConv(60, 30)
    self.conv3 = ChebConv(30, 60, 1)
    self.conv4 = GATConv(60, 120)


  def forward(self, x, edge_index):
    h_5 = self.conv1(x, edge_index)
    h_6 = self.conv2(h_5, edge_index)
    h_7 = self.conv3(h_6, edge_index)
    h_f = self.conv4(h_7, edge_index)

    return h_f


criterion = nn.MSELoss() 
# Number of epochs
num_epochs = 10

# Queue for sharing outputs between models
manager = multiprocessing.Manager()
output_queue1 = manager.Queue()
output_queue2 = manager.Queue()

# Lock to synchronize access to the queues
queue_lock = multiprocessing.Lock()


# def parallel_forward(model, x, edge_index, output_queue):
#     with torch.no_grad():
#         output = model(x, edge_index)

#     # Put the output into the queue
#     output_queue.put(output)

# # Function for backpropagation
# def backpropagation(model, optimizer, criterion, output_queue):
#     for epoch in range(num_epochs):
#         # Get the latest output from the other model
#         other_model_output = output_queue.get()

#         # Forward pass with the other model's output
#         current_output = model(x, edge_index)
#         loss = criterion(current_output, other_model_output)

#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Print loss at each epoch
#         print(f'Epoch {epoch + 1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

def parallel_forward(model, x, edge_index, output_queue):
    with torch.no_grad():
        output = model(x, edge_index)

    # Put the model parameters into the queue
    output_queue.put([param.cpu().detach().numpy() for param in model.parameters()])

# Function for backpropagation
def backpropagation(model, optimizer, criterion, output_queue):
    for epoch in range(num_epochs):
        # Get the latest model parameters from the other model
        other_model_params = output_queue.get()

        # Update the model parameters
        for param, other_param in zip(model.parameters(), other_model_params):
            param.data = torch.from_numpy(other_param)

        # Forward pass with the updated model
        current_output = model(x, edge_index)
        loss = criterion(current_output, target)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch + 1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


# Create instances of the models
model1 = GAE1()
model2 = GAE2()

# Optimizers for each model
optimizer1 = optim.SGD(model1.parameters(), lr=0.01)
optimizer2 = optim.SGD(model2.parameters(), lr=0.01)


with ProcessPoolExecutor() as executor:
    print("Submitting futures...")
    future1 = executor.submit(parallel_forward, model1, x, edge_index, output_queue1)
    future2 = executor.submit(parallel_forward, model2, x, edge_index, output_queue2)
    print("Futures submitted.")

    print("Waiting for results...")
    output1 = future1.result()
    output2 = future2.result()
    print("Results received.")

    # Backpropagation
    print("Starting backpropagation...")
    executor.submit(backpropagation, model1, optimizer1, criterion, output_queue2)
    executor.submit(backpropagation, model2, optimizer2, criterion, output_queue1)
    print("Backpropagation submitted.")

Submitting futures...
Futures submitted.
Waiting for results...


Process ForkProcess-624:
Process ForkProcess-607:
Process ForkProcess-628:
Traceback (most recent call last):
Process ForkProcess-622:
Process ForkProcess-618:
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Process ForkProcess-611:
Traceback (most recent call last):
Process ForkProcess-621:
Process ForkProcess-616:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkProcess-603:
Process ForkProcess-623:
Process ForkProcess-608:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Process ForkProcess-627:
Process ForkProcess-625:
Process ForkProcess-629:
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Process ForkProcess-630:
  File 

KeyboardInterrupt: 

In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
import multiprocessing
from concurrent.futures import ProcessPoolExecutor

def parallel_forward(model, x, edge_index, shared_params):
    with torch.no_grad():
        model.load_state_dict(dict(shared_params))

        output = model(x, edge_index)

    # Put the model parameters into the shared_params dict
    shared_params.update(model.state_dict())

# Function for backpropagation
def backpropagation(model, optimizer, criterion, shared_params):
    for epoch in range(num_epochs):
        # Update the model parameters
        model.load_state_dict(dict(shared_params))

        # Forward pass with the updated model
        current_output = model(x, edge_index)
        loss = criterion(current_output, target)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update the shared_params dict with the new model parameters
        shared_params.update(model.state_dict())

        # Print loss at each epoch
        print(f'Epoch {epoch + 1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

# Rest of your code...

# Use a shared_dict for storing model parameters
shared_params = multiprocessing.Manager().dict()

# Use the ProcessPoolExecutor
with ProcessPoolExecutor() as executor:
    # Submit futures for parallel forward pass
    future1 = executor.submit(parallel_forward, model1, x, edge_index, shared_params)
    future2 = executor.submit(parallel_forward, model2, x, edge_index, shared_params)

    # Wait for the results
    future1.result()
    future2.result()

    # Backpropagation
    executor.submit(backpropagation, model1, optimizer1, criterion, shared_params)
    executor.submit(backpropagation, model2, optimizer2, criterion, shared_params)


RuntimeError: Error(s) in loading state_dict for GAE1:
	Missing key(s) in state_dict: "conv1.bias", "conv1.lins.0.weight", "conv2.att_src", "conv2.att_dst", "conv2.bias", "conv2.lin_src.weight", "conv2.lin_dst.weight", "conv3.bias", "conv3.lins.0.weight", "conv4.att_src", "conv4.att_dst", "conv4.bias", "conv4.lin_src.weight", "conv4.lin_dst.weight". 

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import multiprocessing
from concurrent.futures import ProcessPoolExecutor
import torch
from torch.nn import Linear
from torch_geometric.nn import GATConv, GATv2Conv, GCNConv, TransformerConv, AGNNConv, FusedGATConv, ChebConv, SAGEConv, GraphConv, CuGraphGATConv, TAGConv, GMMConv
from scipy.stats import linregress

class GAE1(torch.nn.Module):
  def __init__(self):
    super(GAE1, self).__init__()
    torch.manual_seed(42)

    self.conv1 = ChebConv(120, 60, 1)
    self.conv2 = GATConv(60, 30)
    self.conv3 = ChebConv(30, 60, 1)
    self.conv4 = GATConv(60, 120)


  def forward(self, x, edge_index):
    h_1 = self.conv1(x, edge_index)
    h_2 = self.conv2(h_1, edge_index)
    h_3 = self.conv3(h_2, edge_index)
    h_a = self.conv4(h_3, edge_index)

    return h_a


class GAE2(torch.nn.Module):
  def __init__(self):
    super(GAE2, self).__init__()
    torch.manual_seed(42)

    self.conv1 = ChebConv(120, 60, 1)
    self.conv2 = GATConv(60, 30)
    self.conv3 = ChebConv(30, 60, 1)
    self.conv4 = GATConv(60, 120)


  def forward(self, x, edge_index):
    h_5 = self.conv1(x, edge_index)
    h_6 = self.conv2(h_5, edge_index)
    h_7 = self.conv3(h_6, edge_index)
    h_f = self.conv4(h_7, edge_index)

    return h_f
      
criterion = nn.MSELoss() 
# Number of epochs
num_epochs = 10

# Dummy data (replace with your actual data)
x = torch.randn(120, 10)
edge_index = torch.randint(0, 120, (2, 500))

# Queue for sharing outputs between models
output_queue1 = multiprocessing.Queue()
output_queue2 = multiprocessing.Queue()

# Lock to synchronize access to the queues
queue_lock = multiprocessing.Lock()

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, output_queue):
    output = model(x, edge_index)
    
    # Put the output into the queue
    with queue_lock:
        output_queue.put(output)

# Function for backpropagation
def backpropagation(model, optimizer, criterion, output_queue):
    for epoch in range(num_epochs):
        # Get the latest output from the other model
        with queue_lock:
            other_model_output = output_queue.get()

        # Forward pass with the other model's output
        current_output = model(x, edge_index)
        loss = criterion(current_output, other_model_output)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

# Create instances of the models
model1 = GAE1()
model2 = GAE2()

# Optimizers for each model
optimizer1 = optim.SGD(model1.parameters(), lr=0.01)
optimizer2 = optim.SGD(model2.parameters(), lr=0.01)

# Parallel forward propagation
with ProcessPoolExecutor() as executor:
    future1 = executor.submit(parallel_forward, model1, x, edge_index, output_queue1)
    future2 = executor.submit(parallel_forward, model2, x, edge_index, output_queue2)

    output1 = future1.result()
    output2 = future2.result()

    # Backpropagation
    executor.submit(backpropagation, model1, optimizer1, criterion, output_queue2)
    executor.submit(backpropagation, model2, optimizer2, criterion, output_queue1)


RuntimeError: Queue objects should only be shared between processes through inheritance

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import multiprocessing

import torch
from torch.nn import Linear
from torch_geometric.nn import GATConv, GATv2Conv, GCNConv, TransformerConv, AGNNConv, FusedGATConv, ChebConv, SAGEConv, GraphConv, CuGraphGATConv, TAGConv, GMMConv
from scipy.stats import linregress

class GAE1(torch.nn.Module):
  def __init__(self):
    super(GAE1, self).__init__()
    torch.manual_seed(42)

    self.conv1 = ChebConv(120, 60, 1)
    self.conv2 = GATConv(60, 30)
    self.conv3 = ChebConv(30, 60, 1)
    self.conv4 = GATConv(60, 120)


  def forward(self, x, edge_index):
    h_1 = self.conv1(x, edge_index)
    h_2 = self.conv2(h_1, edge_index)
    h_3 = self.conv3(h_2, edge_index)
    h_a = self.conv4(h_3, edge_index)

    return h_a


class GAE2(torch.nn.Module):
  def __init__(self):
    super(GAE2, self).__init__()
    torch.manual_seed(42)

    self.conv1 = ChebConv(120, 60, 1)
    self.conv2 = GATConv(60, 30)
    self.conv3 = ChebConv(30, 60, 1)
    self.conv4 = GATConv(60, 120)


  def forward(self, x, edge_index):
    h_5 = self.conv1(x, edge_index)
    h_6 = self.conv2(h_5, edge_index)
    h_7 = self.conv3(h_6, edge_index)
    h_f = self.conv4(h_7, edge_index)

    return h_f


criterion = nn.MSELoss() 
# Number of epochs
num_epochs = 10

# Queue for sharing outputs between models
output_queue1 = multiprocessing.Queue()
output_queue2 = multiprocessing.Queue()

# Lock to synchronize access to the queues
queue_lock = multiprocessing.Lock()

# Function for parallel forward propagation
def parallel_forward(model, x, edge_index, output_queue):
    output = model(x, edge_index)
    
    # Put the output into the queue
    with queue_lock:
        output_queue.put((output))

# Function for backpropagation
def backpropagation_aging(model, optimizer, criterion, output_queue):
    for epoch in range(num_epochs):
        # Get the latest output from the other model
        with queue_lock:
            h_f = output_queue.get()

        # Forward pass with the other model's output
        h_a = model(x, edge_index)
        # loss = criterion(output, target1) + criterion(other_model_output, other_model_target)
        loss = criterion(h_a+h_f, x) + 10 * torch.std(torch.diff(h_a))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


def backpropagation_fluctuation(model, optimizer, criterion, output_queue):
    for epoch in range(num_epochs):
        # Get the latest output from the other model
        with queue_lock:
            h_a = output_queue.get()

        # Forward pass with the other model's output
        output = model(x, edge_index)
        # loss = criterion(output, target1) + criterion(other_model_output, other_model_target)
        loss = criterion(h_a+h_f, x) + torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(h_f, 0, 1))[1])) +\
        torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(h_f, E.T).long(), L.long()), torch.matmul(E, h_f.T).long()), 0))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')


# Create instances of the models
model1 = GAE1()
model2 = GAE2()

# Optimizers for each model
optimizer1 = optim.SGD(model1.parameters(), lr=0.01)
optimizer2 = optim.SGD(model2.parameters(), lr=0.01)

# Parallel forward propagation
with ProcessPoolExecutor() as executor:
    future1 = executor.submit(parallel_forward, model1, x, edge_index)
    future2 = executor.submit(parallel_forward, model2, x, edge_index)

    output1 = future1.result()
    output2 = future2.result()

    # Backpropagation
    executor.submit(backpropagation, (model1, optimizer1, nn.CrossEntropyLoss(), output2))
    executor.submit(backpropagation, (model2, optimizer2, nn.CrossEntropyLoss(), output1))

TypeError: parallel_forward() missing 1 required positional argument: 'output_queue'

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import multiprocessing

# Define the models
class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.fc1 = nn.Linear(10, 5)

    def forward(self, x):
        return self.fc1(x)

class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.fc1 = nn.Linear(10, 5)

    def forward(self, x):
        return self.fc1(x)

# Dummy data (replace with your actual data)
input_data = torch.randn(5, 10)
target1 = torch.randint(0, 5, (5,))
target2 = torch.randint(0, 5, (5,))

# Number of epochs
num_epochs = 10

# Queue for sharing outputs between models
output_queue1 = multiprocessing.Queue()
output_queue2 = multiprocessing.Queue()

# Lock to synchronize access to the queues
queue_lock = multiprocessing.Lock()

# Function for parallel forward propagation
def parallel_forward(model, input_data, output_queue, target):
    output = model(input_data)
    
    # Put the output into the queue
    with queue_lock:
        output_queue.put((output, target))

# Function for backpropagation
def backpropagation(model, optimizer, criterion, output_queue, other_model_target):
    for epoch in range(num_epochs):
        # Get the latest output from the other model
        with queue_lock:
            other_model_output, other_model_target = output_queue.get()

        # Forward pass with the other model's output
        output = model(input_data)
        loss = criterion(output, target1) + criterion(other_model_output, other_model_target)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print loss at each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Model: {model.__class__.__name__}, Loss: {loss.item()}')

# Create instances of the models
model1 = Model1()
model2 = Model2()

# Optimizers for each model
optimizer1 = optim.SGD(model1.parameters(), lr=0.01)
optimizer2 = optim.SGD(model2.parameters(), lr=0.01)

# Create processes for parallel learning
process1 = multiprocessing.Process(target=parallel_forward, args=(model1, input_data, output_queue1, target1))
process2 = multiprocessing.Process(target=parallel_forward, args=(model2, input_data, output_queue2, target2))

# Start the processes
process1.start()
process2.start()

# Backpropagation processes
backprop_process1 = multiprocessing.Process(target=backpropagation, args=(model1, optimizer1, nn.CrossEntropyLoss(), output_queue2, target2))
backprop_process2 = multiprocessing.Process(target=backpropagation, args=(model2, optimizer2, nn.CrossEntropyLoss(), output_queue1, target1))

# Start the backpropagation processes
backprop_process1.start()
backprop_process2.start()

# Wait for all processes to finish
process1.join()
process2.join()
backprop_process1.join()
backprop_process2.join()

Process Process-133:
Process Process-134:
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/job.793795.hpc/ipykernel_55350/199444358.py", line 51, in backpropagation
    other_model_output, other_model_target = output_queue.get()
  File "/tmp/job.793795.hpc/ipykernel_55350/199444358.py", line 50, in backpropagation
    with queue_lock:
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 216, in recv_bytes
   

In [ ]:
import multiprocessing
import torch
import torch.nn as nn
import torch.optim as optim
import time
from torch.nn import Linear
from torch_geometric.nn import ChebConv, GATConv
from scipy.stats import linregress
from concurrent.futures import ThreadPoolExecutor

# Number of epochs for training
num_epochs = 100

# Create two instances of the GAE models
model1 = GAE1()
model2 = GAE2()

if __name__ == '__main__':
    # Create a ThreadPoolExecutor
    with ThreadPoolExecutor() as executor:
        # Submit tasks to the ThreadPoolExecutor
        future1 = executor.submit(train_GAE1, model1, x, edge_index, num_epochs)
        future2 = executor.submit(train_GAE2, model2, x, edge_index, num_epochs)

        # Wait for both tasks to finish
        concurrent.futures.wait([future1, future2])
       # x = self.relu(x)
#         x = self.fc2(x)
#         x = self.relu(x)
#         x = self.fc3(x)
#         return x

# # Function to train a model
# def train_model(model, data, epochs):
#     criterion = nn.MSELoss()
#     optimizer = optim.SGD(model.parameters(), lr=0.01)
#     start_time = time.time()

#     for epoch in range(epochs):
#         inputs, targets = data
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         print(f"Model {model.__class__.__name__}, Epoch {epoch+1}, Loss: {loss.item()}")

#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print(f"Training time for {model.__class__.__name__}: {elapsed_time} seconds")

# Create two instances of the complex neural network model
model1 = GAE1()
model2 = GAE2()

# # Dummy data for training
# data1 = (torch.randn(5, 10), torch.randn(5, 1))
# data2 = (torch.randn(5, 10), torch.randn(5, 1))

# Number of epochs for training
num_epochs = 100

# # start_time = time.time()
# # Create two processes for training each model
# process1 = multiprocessing.Process(target=train_GAE1, args=(model1, x, edge_index, num_epochs))
# process2 = multiprocessing.Process(target=train_GAE2, args=(model2, x, edge_index, num_epochs))

# # Start the processes
# process1.start()
# process2.start()

# # Wait for both processes to finish
# process1.join()
process2.join()



if __name__ == '__main__':
    # Create two processes for training each model
    process1 = multiprocessing.Process(target=train_GAE1, args=(model1, x, edge_index, num_epochs))
    process2 = multiprocessing.Process(target=train_GAE2, args=(model2, x, edge_index, num_epochs))

    # Start the processes
    process1.start()
    process2.start()

    # Wait for both processes to finish
    process1.join()
    process2.join()

In [ ]:
from sklearn.metrics import mean_squared_error
import torch
# import tensorflow as tf
import torch.nn.functional as F

Lambda1 = 0
Lambda2 = 100
Lambda3 = 5

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
start_time = time.time()

def train(x, edge_index, Lambda1, Lambda2):
  optimizer.zero_grad()
  h_a, h_f = model(x, edge_index)
  loss = criterion(h_a+h_f, x)  + \
  10*torch.std(torch.diff(h_a)) +\
  Lambda1 * torch.var(h_a) +\
  Lambda2 * torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(h_f, 0, 1))[1])) +\
  Lambda3 * torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(h_f, E.T).long(), L.long()), torch.matmul(E, h_f.T).long()), 0))
  loss.backward()
  optimizer.step()
  return loss, criterion(h_a+h_f, x), h_a, h_f

epochs = range(1, 500)
losses = []
embeddings = []
min_loss = 1000
for epoch in epochs:
  loss, loss_2, h_a, h_f= train(x, edge_index, Lambda1, Lambda2)
  losses.append(loss)
  # print(f"Epoch: {epoch}\tLoss: {loss:.4f}")
  # if loss_2 < 100:
  #   break

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

In [11]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GATConv, GATv2Conv, GCNConv, TransformerConv, AGNNConv, FusedGATConv, ChebConv, SAGEConv, GraphConv, CuGraphGATConv, TAGConv, GMMConv
from scipy.stats import linregress

class GCN(torch.nn.Module):
  def __init__(self):
    super(GCN, self).__init__()
    torch.manual_seed(42)

    self.conv1 = GATConv(120, 60)
    self.conv2 = GATConv(60, 120)
    self.conv3 = GATConv(120, 60)
    self.conv4 = GATConv(60, 120)


  def forward(self, x, edge_index):


    h_1 = self.conv1(x, edge_index)
    h_a = self.conv2(h_1, edge_index)


    h_2 = self.conv3(x, edge_index)
    h_f = self.conv4(h_2, edge_index)

    return h_a, h_f

model = GCN()
print(model)

GCN(
  (conv1): GATConv(120, 60, heads=1)
  (conv2): GATConv(60, 120, heads=1)
  (conv3): GATConv(120, 60, heads=1)
  (conv4): GATConv(60, 120, heads=1)
)


In [15]:
import torch
import torch.nn as nn
import dgl
from dgl.nn.pytorch import ChebConv, GATConv

class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30, 1)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120, 1)

        self.conv5 = ChebConv(120, 60, 1)
        self.conv6 = GATConv(60, 30, 1)
        self.conv7 = ChebConv(30, 60, 1)
        self.conv8 = GATConv(60, 120, 1)

    def forward(self, g, x):
        h_1 = self.conv1(g, x)
        h_2 = self.conv2(g, h_1)
        h_3 = self.conv3(g, h_2)
        h_a = self.conv4(g, h_3)

        h_5 = self.conv5(g, x)
        h_6 = self.conv6(g, h_5)
        h_7 = self.conv7(g, h_6)
        h_f = self.conv8(g, h_7)

        return h_a, h_f

model = GCN()
print(model)

OSError: libcudart.so.11.0: cannot open shared object file: No such file or directory

In [21]:
import torch
import torch.nn as nn
import dgl
from dgl.nn.pytorch import ChebConv, GATConv, DotGatConv

class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(42)

        self.conv1 = GATConv(120, 60, 1)
        self.conv2 = GATConv(60, 120, 1)
        self.conv3 = GATConv(120, 60, 1)
        self.conv4 = GATConv(60, 120, 1)

    def forward(self, blocks, x):
        h_1 = self.conv1(blocks[0], x)
        h_a = self.conv2(blocks[1], h_1)
        h_2 = self.conv3(blocks[0], x)
        h_f = self.conv4(blocks[1], h_2)

        return h_a, h_f

model = GCN()
print(model)


GCN(
  (conv1): GATConv(
    (fc): Linear(in_features=120, out_features=60, bias=False)
    (feat_drop): Dropout(p=0.0, inplace=False)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (leaky_relu): LeakyReLU(negative_slope=0.2)
  )
  (conv2): GATConv(
    (fc): Linear(in_features=60, out_features=120, bias=False)
    (feat_drop): Dropout(p=0.0, inplace=False)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (leaky_relu): LeakyReLU(negative_slope=0.2)
  )
  (conv3): GATConv(
    (fc): Linear(in_features=120, out_features=60, bias=False)
    (feat_drop): Dropout(p=0.0, inplace=False)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (leaky_relu): LeakyReLU(negative_slope=0.2)
  )
  (conv4): GATConv(
    (fc): Linear(in_features=60, out_features=120, bias=False)
    (feat_drop): Dropout(p=0.0, inplace=False)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (leaky_relu): LeakyReLU(negative_slope=0.2)
  )
)


In [50]:
import torch
import torch.nn as nn
import dgl
from dgl.nn.pytorch import ChebConv, GATConv, GraphConv

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        torch.manual_seed(42)

        self.conv1 = ChebConv(120, 60, 1)
        self.conv2 = GATConv(60, 30, 1)
        self.conv3 = ChebConv(30, 60, 1)
        self.conv4 = GATConv(60, 120, 1)

        self.conv5 = ChebConv(120, 60, 1)
        self.conv6 = GATConv(60, 30, 1)
        self.conv7 = ChebConv(30, 60, 1)
        self.conv8 = GATConv(60, 120, 1)

    def forward(self, g, in_feat):
        # h_1 = self.conv1(g, in_feat)
        
        # h_a = self.conv2(g, h_1).relu()
        # h_2 = self.conv3(g, in_feat)
        # h_f = self.conv4(g, h_2).relu()

        h_1 = self.conv1(g, x)
        h_2 = self.conv2(g, h_1).relu()
        h_3 = self.conv3(g, h_2)
        h_a = self.conv4(g, h_3).relu()

        h_5 = self.conv5(g, x)
        h_6 = self.conv6(g, h_5).relu()
        h_7 = self.conv7(g, h_6)
        h_f = self.conv8(g, h_7).relu()


        return h_a, h_f

model = Model()
print(model)

Model(
  (conv1): ChebConv(
    (linear): Linear(in_features=120, out_features=60, bias=True)
  )
  (conv2): GATConv(
    (fc): Linear(in_features=60, out_features=30, bias=False)
    (feat_drop): Dropout(p=0.0, inplace=False)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (leaky_relu): LeakyReLU(negative_slope=0.2)
  )
  (conv3): ChebConv(
    (linear): Linear(in_features=30, out_features=60, bias=True)
  )
  (conv4): GATConv(
    (fc): Linear(in_features=60, out_features=120, bias=False)
    (feat_drop): Dropout(p=0.0, inplace=False)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (leaky_relu): LeakyReLU(negative_slope=0.2)
  )
  (conv5): ChebConv(
    (linear): Linear(in_features=120, out_features=60, bias=True)
  )
  (conv6): GATConv(
    (fc): Linear(in_features=60, out_features=30, bias=False)
    (feat_drop): Dropout(p=0.0, inplace=False)
    (attn_drop): Dropout(p=0.0, inplace=False)
    (leaky_relu): LeakyReLU(negative_slope=0.2)
  )
  (conv7): ChebConv(
    (linear): L

In [ ]:
import torch
import torch.nn as nn
import dgl
from dgl.nn.pytorch import ChebConv, GATConv, GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GATConv(120, 60, 1)
        self.conv2 = GATConv(60, 120, 1)
        self.conv3 = GATConv(120, 60, 1)
        self.conv4 = GATConv(60, 120, 1)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)


In [11]:
import dgl

sampler = dgl.dataloading.MultiLayerFullNeighborSampler(2)


OSError: libcudart.so.11.0: cannot open shared object file: No such file or directory

In [12]:
import torch

print(torch.version.cuda)

pip install dgl-cu121


12.1


In [13]:
pip install dgl-cu121

ERROR: Could not find a version that satisfies the requirement dgl-cu121 (from versions: none)
ERROR: No matching distribution found for dgl-cu121

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
import dgl
import torch
from torch_geometric.data import Data


# PyG Data object
pyg_data = Data(x=x, edge_index=edge_index)

# Extract PyG graph information
num_nodes = pyg_data.num_nodes
num_edges = pyg_data.num_edges
src, dst = pyg_data.edge_index

# Create DGL graph
dgl_graph = dgl.graph((src, dst), num_nodes=num_nodes)

# Optional: Set node features for DGL graph
dgl_graph.ndata['x'] = pyg_data.x

In [26]:
dgl_graph

Graph(num_nodes=100, num_edges=2746,
      ndata_schemes={'x': Scheme(shape=(120,), dtype=torch.float32)}
      edata_schemes={})

In [15]:
dataloader = dgl.dataloading.DataLoader(
    dgl_graph, dgl_graph.nodes().long(), sampler,
    batch_size=25,
    shuffle=False,
    drop_last=False,
    num_workers=2)

In [27]:
from sklearn.metrics import mean_squared_error
import torch
# import tensorflow as tf
import torch.nn.functional as F

Lambda1 = 0
Lambda2 = 100
Lambda3 = 5

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
start_time = time.time()

def train(x, edge_index, Lambda1, Lambda2):
  optimizer.zero_grad()
  h_a, h_f = model(x, edge_index)
  loss = criterion(h_a+h_f, x)  + \
  10*torch.std(torch.diff(h_a)) +\
  Lambda1 * torch.var(h_a) +\
  Lambda2 * torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(h_f, 0, 1))[1])) +\
  Lambda3 * torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(h_f, E.T).long(), L.long()), torch.matmul(E, h_f.T).long()), 0))

  # print(criterion(h_a + h_f, x))
  # print(torch.std(torch.diff(h_a)))
  # print(torch.var(h_a))
  # print(torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(h_f, 0, 1))[1])))
  # print(torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(h_f, E.T).long(), L.long()), torch.matmul(E, h_f.T).long()), 0)))

  loss.backward()
  optimizer.step()
  return loss, criterion(h_a+h_f, x), h_a, h_f

epochs = range(1, 500)
losses = []
embeddings = []
min_loss = 1000
for epoch in epochs:
  loss, loss_2, h_a, h_f= train(x, edge_index, Lambda1, Lambda2)
  losses.append(loss)
  # print(f"Epoch: {epoch}\tLoss: {loss:.4f}")
  # if loss_2 < 100:
  #   break

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

Training time: 7.413655757904053 seconds


In [55]:
  print(criterion(h_a + h_f, x))
  print(torch.std(torch.diff(h_a)))
  print(torch.var(h_a))
  print(torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(h_f, 0, 1))[1])))
  print(torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(h_f, E.T).long(), L.long()), torch.matmul(E, h_f.T).long()), 0)))


tensor(9.6694, grad_fn=<MseLossBackward0>)
tensor(0.5692, grad_fn=<StdBackward0>)
tensor(6.3497, grad_fn=<VarBackward0>)
tensor(0.0146, grad_fn=<SumBackward0>)
tensor(13080)


In [ ]:
blocks[1].dstdata['x'].shape

In [13]:
# device = torch.device("cuda") if torch.cuda.is_available() \
# else torch.device("cpu")

# model = model.to(device)
# m = m.to(device)
# E = E.to(device)
# L = L.to(device)
# # # Move the input features to the same device as the model
# # input_features = input_features.to(device)

# for input_nodes, output_nodes, blocks in dataloader:
#     blocks = [b.to(device) for b in blocks]
#     input_features = blocks[0].srcdata['x'].to(device)
#     ground_truth = blocks[1].dstdata['x'].to(device)
#     h_a, h_f = model(blocks, input_features)
#     loss = criterion(torch.squeeze(h_a)+torch.squeeze(h_f), ground_truth)  + \
#     10*torch.std(torch.diff(h_a)) +\
#     Lambda1 * torch.var(h_a) +\
#     Lambda2 * torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(torch.squeeze(h_f), 0, 1))[1])) +\
#     Lambda3 * torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(torch.squeeze(h_f), E.T).float(), L.float()), torch.matmul(E, torch.squeeze(h_f).T).float()), 0))
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     print(loss)

In [64]:
dgl_graph.ndata["x"]

tensor([[48.5786, 46.4354, 50.6477,  ..., 33.6993, 37.9428, 47.2802],
        [48.5926, 46.4799, 50.7236,  ..., 33.2002, 37.3683, 46.5494],
        [48.6029, 46.5166, 50.7899,  ..., 32.7940, 36.9030, 45.9515],
        ...,
        [41.7236, 44.8196, 43.9540,  ..., 40.6397, 42.6384, 40.6262],
        [41.7287, 44.8377, 43.9773,  ..., 40.4663, 42.4543, 40.4481],
        [41.7256, 44.8226, 43.9507,  ..., 40.6659, 42.6745, 40.6584]])

In [12]:
# Assuming 'dataloader' is your PyTorch DataLoader
Lambda1 = 0
Lambda2 = 100
Lambda3 = 5

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50  # You can adjust the number of epochs
# device = torch.device("cuda") if torch.cuda.is_available() \
# else torch.device("cpu")

# model = model.to(device)
# m = m.to(device)
# E = E.to(device)
# L = L.to(device)
# dgl_graph = dgl_graph.to(device)


def train(x, edge_index, Lambda1, Lambda2):
  optimizer.zero_grad()
  features = dgl_graph.ndata["x"]
  ground_truth = dgl_graph.ndata["x"]
  h_a, h_f = model(dgl_graph, features)
  loss = criterion(torch.squeeze(h_a)+torch.squeeze(h_f), ground_truth)  + \
  10*torch.std(torch.diff(torch.squeeze(h_a))) +\
  0 * torch.var(torch.squeeze(h_a)) +\
  100 * torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(torch.squeeze(h_f), 0, 1))[1])) +\
  5 * torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(torch.squeeze(h_f), E.T).long(), L.long()), torch.matmul(E, torch.squeeze(h_f).T).long()), 0))
  print(criterion(torch.squeeze(h_a) + torch.squeeze(h_f), x))
  print(torch.std(torch.diff(torch.squeeze(h_a))))
  print(torch.var(torch.squeeze(h_a)))
  print(torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(torch.squeeze(h_f), 0, 1))[1])))
  print(torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(torch.squeeze(h_f), E.T).long(), L.long()), torch.matmul(E, torch.squeeze(h_f).T).long()), 0)))

  loss.backward()
  optimizer.step()
  return loss, criterion(h_a+h_f, x), h_a, h_f

epochs = range(1, 500)
losses = []
embeddings = []
min_loss = 1000
for epoch in epochs:
  loss, loss_2, h_a, h_f= train(x, edge_index, Lambda1, Lambda2)
  losses.append(loss)
  print(f"Epoch: {epoch}\tLoss: {loss:.4f}")

# for epoch in range(num_epochs):
#     losses = []
#     for input_nodes, output_nodes, blocks in dataloader:
#         # Move blocks, input_features, and ground_truth to device
#         blocks = [b.to(device) for b in blocks]
#         input_features = blocks[0].srcdata['x'].to(device)
#         ground_truth = blocks[-1].dstdata['x'].to(device)

#         # Forward pass
#         h_a, h_f = model(blocks, input_features)

#         # Compute loss
#         loss = criterion(torch.squeeze(h_a) + torch.squeeze(h_f), ground_truth) + \
#                10 * torch.std(torch.diff(h_a)) + \
#                Lambda1 * torch.var(h_a) + \
#                Lambda2 * torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)).float(),
#                                                                    torch.transpose(m, 0, 1).float()),
#                                                         torch.transpose(torch.squeeze(h_f), 0, 1).float())[1])) + \
#                Lambda3 * torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(torch.squeeze(h_f), E.T).float(),
#                                                                         L.float()),
#                                                              torch.matmul(E, torch.squeeze(h_f).T).float()), 0))

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         losses.append(loss.item())

#     print(f"Epoch {epoch + 1}, Loss: {np.mean(losses)}")

# print("Training complete!")


tensor(2485.2222, grad_fn=<MseLossBackward0>)
tensor(30.2983, grad_fn=<StdBackward0>)
tensor(458.8483, grad_fn=<VarBackward0>)
tensor(0.4608, grad_fn=<SumBackward0>)
tensor(108331)
Epoch: 1	Loss: 544489.2500
tensor(2131.1416, grad_fn=<MseLossBackward0>)
tensor(23.0252, grad_fn=<StdBackward0>)
tensor(272.3502, grad_fn=<VarBackward0>)
tensor(4.5825, grad_fn=<SumBackward0>)
tensor(204195)
Epoch: 2	Loss: 1023794.6250
tensor(1884.9075, grad_fn=<MseLossBackward0>)
tensor(17.6219, grad_fn=<StdBackward0>)
tensor(160.7885, grad_fn=<VarBackward0>)
tensor(3.8973, grad_fn=<SumBackward0>)
tensor(161619)
Epoch: 3	Loss: 810545.8750
tensor(1747.4886, grad_fn=<MseLossBackward0>)
tensor(13.8711, grad_fn=<StdBackward0>)
tensor(99.9352, grad_fn=<VarBackward0>)
tensor(2.0982, grad_fn=<SumBackward0>)
tensor(95546)
Epoch: 4	Loss: 479826.0312
tensor(1677.5245, grad_fn=<MseLossBackward0>)
tensor(11.3457, grad_fn=<StdBackward0>)
tensor(66.2654, grad_fn=<VarBackward0>)
tensor(0.2143, grad_fn=<SumBackward0>)
tens

In [16]:
  print(criterion(torch.squeeze(h_a) + torch.squeeze(h_f), x))
  print(torch.std(torch.diff(torch.squeeze(h_a))))
  print(torch.var(torch.squeeze(h_a)))
  print(torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(torch.squeeze(h_f), 0, 1))[1])))
  print(torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(torch.squeeze(h_f), E.T).long(), L.long()), torch.matmul(E, torch.squeeze(h_f).T).long()), 0)))


tensor(13.3835, grad_fn=<MseLossBackward0>)
tensor(0.3352, grad_fn=<StdBackward0>)
tensor(6.9074, grad_fn=<VarBackward0>)
tensor(0.0189, grad_fn=<SumBackward0>)
tensor(2500)


In [47]:
g = dgl.graph(([0,1,2,3,2,5], [1,2,3,4,0,3]))
g = dgl.add_self_loop(g)
feat = torch.ones(6, 10)
gatconv1 = GATConv(10, 2, num_heads=1)
gatconv2 = GATConv(2, 10, num_heads=1)
res = gatconv1(g, feat)
f = gatconv2(g, res)
f.shape

torch.Size([6, 1, 1, 10])

In [45]:
feat.shape

torch.Size([6, 10])

In [ ]:
import time

# ... (previous code)

# Lists to store loss values for plotting
train_losses = []

# Start the timer
start_time = time.time()

for epoch in range(num_epochs):
    losses = []
    for input_nodes, output_nodes, blocks in dataloader:
        # ... (rest of your code)

    # Calculate the mean loss for the epoch and store it
    epoch_loss = np.mean(losses)
    train_losses.append(epoch_loss)

    print(f"Epoch {epoch + 1}, Loss: {epoch_loss}")

# Stop the timer
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Training time: {elapsed_time} seconds")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

num_epochs = 100  # You can adjust the number of epochs
device = torch.device("cuda") if torch.cuda.is_available() \
else torch.device("cpu")

model = model.to(device)
m = m.to(device)
E = E.to(device)
L = L.to(device)

# Lists to store loss values for plotting
train_losses = []
start_time = time.time()

for epoch in range(num_epochs):
    losses = []
    for input_nodes, output_nodes, blocks in dataloader:
        # ... (rest of your code)
                # Move blocks, input_features, and ground_truth to device
        blocks = [b.to(device) for b in blocks]
        input_features = blocks[0].srcdata['x'].to(device)
        ground_truth = blocks[-1].dstdata['x'].to(device)

        # Forward pass
        h_a, h_f = model(blocks, input_features)

        # Compute loss
        loss = criterion(torch.squeeze(h_a) * torch.squeeze(h_f), ground_truth) + \
               10 * torch.std(torch.diff(h_a)) + \
               Lambda1 * torch.var(h_a) + \
               Lambda2 * torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)).float(),
                                                                   torch.transpose(m, 0, 1).float()),
                                                        torch.transpose(torch.squeeze(h_f), 0, 1).float())[1])) + \
               Lambda3 * torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(torch.squeeze(h_f), E.T).float(),
                                                                        L.float()),
                                                             torch.matmul(E, torch.squeeze(h_f).T).float()), 0))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Append the current batch loss to the losses list
        losses.append(loss.item())

    # Calculate the mean loss for the epoch and store it
    epoch_loss = np.mean(losses)
    train_losses.append(epoch_loss)

    print(f"Epoch {epoch + 1}, Loss: {epoch_loss}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Training time: {elapsed_time} seconds")

# Plot the training loss
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

print("Training complete!")


# Multi-GPU Training

In [20]:
import torch

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print("Number of available GPUs:", num_gpus)
else:
    print("CUDA is not available. Make sure you have installed the correct GPU version of PyTorch.")


dgl_graph.create_formats_()

Number of available GPUs: 2


In [31]:
from st_DynGNN import run
devices = torch.device("cuda") if torch.cuda.is_available() \
else torch.device("cpu")

num_epochs = 100

if __name__ == '__main__':
    num_gpus = 1
    import torch.multiprocessing as mp
    mp.spawn(run, args=(list(range(num_gpus)),dgl_graph), nprocs=num_gpus)


/home/rxf131/ondemand/ubuntu2204/python310/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
  0%|          | 0/4 [00:00<?, ?it/s]

tensor([[[[ -32.5790, -179.3676,  -90.9743,  ...,   -3.0696,   42.0174,
             21.1875]]],


        [[[ -30.8580, -177.6286,  -90.0213,  ...,   -3.1443,   43.7388,
             20.7994]]],


        [[[ -32.5045, -189.1380,  -91.3025,  ...,   -6.8868,   28.2495,
             20.4220]]],


        ...,


        [[[ -32.5006, -189.1476,  -91.3071,  ...,   -6.8879,   28.2568,
             20.4181]]],


        [[[ -32.5012, -189.1460,  -91.3064,  ...,   -6.8877,   28.2556,
             20.4187]]],


        [[[ -30.8582, -177.6274,  -90.0215,  ...,   -3.1438,   43.7394,
             20.7995]]]], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[[[  -7.8046, -114.5195,   -8.4159,  ...,  -50.8800,    4.0630,
             63.9066]]],


        [[[  -7.8046, -114.5197,   -8.4156,  ...,  -50.8803,    4.0623,
             63.9067]]],


        [[[ -18.8121, -139.5337,  -14.4212,  ...,  -67.9004,   -4.4609,
             79.1707]]],


        ...,


        [[[  -7.8167, -114.5293,   -8.4

  0%|          | 0/4 [00:00<?, ?it/s]

tensor([[[[-70.5393,  45.6838,  14.9451,  ..., 118.4556, -82.4690,  66.4810]]],


        [[[-67.1827,  46.6613,  13.4075,  ..., 115.4219, -62.3244,  65.0866]]],


        [[[-67.1826,  46.6612,  13.4075,  ..., 115.4219, -62.3243,  65.0866]]],


        ...,


        [[[-67.1826,  46.6613,  13.4075,  ..., 115.4219, -62.3243,  65.0866]]],


        [[[-67.1827,  46.6613,  13.4075,  ..., 115.4219, -62.3244,  65.0866]]],


        [[[-89.7910,  47.1971,   8.6471,  ..., 126.0947, -88.6425,  72.4640]]]],
       device='cuda:0', grad_fn=<AddBackward0>)
tensor([[[[-56.8661,  53.9048,  40.2695,  ..., 135.1146, -63.5645,  21.0893]]],


        [[[-40.7706,  55.9235,  39.4697,  ..., 128.6955, -46.4366,  11.8905]]],


        [[[-40.7712,  55.9238,  39.4697,  ..., 128.6959, -46.4375,  11.8906]]],


        ...,


        [[[-40.7712,  55.9238,  39.4697,  ..., 128.6959, -46.4375,  11.8906]]],


        [[[-42.9424,  55.5081,  42.3046,  ..., 132.7780, -65.7513,  10.4847]]],


        [[[-42.9415, 

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

tensor([[[[39.9030, 65.8259, 81.4105,  ..., 41.1737, 57.0088, 38.4528]]],


        [[[40.4078, 73.7325, 90.6303,  ..., 37.3559, 51.9572, 40.0909]]],


        [[[42.1284, 68.6028, 84.8210,  ..., 40.7923, 54.6320, 36.8044]]],


        ...,


        [[[43.6010, 68.8947, 79.5505,  ..., 40.8360, 51.5567, 32.3070]]],


        [[[39.9068, 65.8607, 81.4402,  ..., 41.1801, 56.9803, 38.4340]]],


        [[[40.4032, 73.7381, 90.6323,  ..., 37.3594, 51.9527, 40.0885]]]],
       device='cuda:0', grad_fn=<AddBackward0>)
tensor([[[[40.0028, 65.5096, 81.7942,  ..., 40.8818, 56.9674, 38.7985]]],


        [[[43.7070, 68.5318, 79.8861,  ..., 40.5329, 51.5893, 32.6962]]],


        [[[40.0022, 65.5091, 81.7934,  ..., 40.8819, 56.9678, 38.7988]]],


        ...,


        [[[43.7061, 68.5329, 79.8865,  ..., 40.5335, 51.5885, 32.6958]]],


        [[[40.0016, 65.5071, 81.7916,  ..., 40.8817, 56.9694, 38.7999]]],


        [[[42.2272, 68.2428, 85.1532,  ..., 40.4906, 54.6500, 37.1745]]]],
       devic

 75%|███████▌  | 3/4 [00:00<00:00, 27.24it/s]

tensor([[[[41.0135, 74.0300, 91.6132,  ..., 37.7646, 53.3115, 41.0549]]],


        [[[40.1750, 65.8433, 82.0581,  ..., 41.5717, 58.1565, 39.4366]]],


        [[[41.0151, 74.0280, 91.6124,  ..., 37.7634, 53.3131, 41.0557]]],


        ...,


        [[[42.5876, 68.8129, 85.5133,  ..., 41.1960, 55.8575, 37.6738]]],


        [[[40.1753, 65.8204, 82.0409,  ..., 41.5660, 58.1741, 39.4487]]],


        [[[40.1708, 65.8126, 82.0328,  ..., 41.5662, 58.1801, 39.4531]]]],
       device='cuda:0', grad_fn=<AddBackward0>)
tensor([[[[41.3187, 66.2366, 82.1314,  ..., 40.9314, 58.3131, 39.7459]]],


        [[[41.3174, 66.2342, 82.1290,  ..., 40.9314, 58.3149, 39.7473]]],


        [[[41.3184, 66.2352, 82.1302,  ..., 40.9312, 58.3142, 39.7467]]],


        ...,


        [[[41.3194, 66.2374, 82.1324,  ..., 40.9313, 58.3125, 39.7455]]],


        [[[41.3188, 66.2369, 82.1317,  ..., 40.9314, 58.3129, 39.7458]]],


        [[[43.7096, 69.1031, 85.4575,  ..., 40.5398, 56.1301, 38.0476]]]],
       devic

100%|██████████| 4/4 [00:00<00:00, 44.65it/s]


In [30]:
from st_DynGNN import run
devices = torch.device("cuda") if torch.cuda.is_available() \
else torch.device("cpu")

dgl_graph.create_formats_()

if __name__ == '__main__':
    num_gpus = 2
    import torch.multiprocessing as mp
    mp.spawn(run, args=(list(range(num_gpus)),dgl_graph), nprocs=num_gpus)

/home/rxf131/ondemand/ubuntu2204/python310/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/rxf131/ondemand/ubuntu2204/python310/scipy/__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.21.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
  0%|          | 0/2 [00:00<?, ?it/s]

tensor([[[[ -44.8861, -210.5856, -106.1942,  ...,  -12.3924,   39.9872,
             30.7661]]],


        [[[ -44.8903, -210.5752, -106.1891,  ...,  -12.3911,   39.9778,
             30.7704]]],


        [[[ -44.8870, -210.5836, -106.1932,  ...,  -12.3922,   39.9854,
             30.7669]]],


        ...,


        [[[ -49.1797, -194.3997, -100.4074,  ...,   -1.7012,   26.8106,
             24.9881]]],


        [[[ -44.8855, -210.5870, -106.1949,  ...,  -12.3926,   39.9886,
             30.7654]]],


        [[[ -30.8586, -177.6261,  -90.0209,  ...,   -3.1436,   43.7385,
             20.8000]]]], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[[[ -30.8580, -177.6286,  -90.0213,  ...,   -3.1443,   43.7388,
             20.7994]]],


        [[[ -31.6060, -178.4310,  -90.4268,  ...,   -3.1302,   42.9495,
             20.9672]]],


        [[[ -32.5050, -189.1369,  -91.3021,  ...,   -6.8867,   28.2487,
             20.4225]]],


        ...,


        [[[ -32.5042, -189.1388,  -91.3

  0%|          | 0/2 [00:00<?, ?it/s]

tensor([[[[ 17.9419, -70.3500,  17.1129,  ..., -13.7609, -61.4273, 118.2391]]],


        [[[ 17.9408, -70.3467,  17.1140,  ..., -13.7602, -61.4290, 118.2397]]],


        [[[ 10.0961, -89.8651,  14.1683,  ..., -26.9861, -78.2340, 140.3002]]],


        ...,


        [[[ 17.9409, -70.3471,  17.1138,  ..., -13.7603, -61.4288, 118.2396]]],


        [[[ 17.9418, -70.3496,  17.1130,  ..., -13.7608, -61.4275, 118.2392]]],


        [[[ 10.0953, -89.8636,  14.1689,  ..., -26.9859, -78.2353, 140.3007]]]],
       device='cuda:0', grad_fn=<AddBackward0>)
tensor([[[[  2.9681, -79.2708,  13.9826,  ..., -15.6563, -86.7673, 130.0384]]],


        [[[ 17.9415, -70.3495,  17.1148,  ..., -13.7617, -61.4313, 118.2404]]],


        [[[ 19.1330, -77.3438,  19.7162,  ..., -18.4334, -81.0505, 122.0236]]],


        ...,


        [[[ 19.1328, -77.3434,  19.7164,  ..., -18.4333, -81.0508, 122.0238]]],


        [[[ 17.9549, -70.3413,  17.1166,  ..., -13.7594, -61.4066, 118.2292]]],


        [[[ 19.1285, 

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

tensor([[[[39.3700, 67.9104, 88.4233,  ..., 34.2668, 54.3662, 45.1807]]],


        [[[41.8950, 64.9113, 84.5337,  ..., 38.8674, 59.1026, 40.0935]]],


        [[[42.0051, 65.0537, 84.8681,  ..., 38.8164, 58.8270, 40.0668]]],


        ...,


        [[[41.8942, 64.9106, 84.5317,  ..., 38.8679, 59.1041, 40.0936]]],


        [[[41.8953, 64.9121, 84.5351,  ..., 38.8673, 59.1013, 40.0933]]],


        [[[41.8953, 64.9126, 84.5359,  ..., 38.8674, 59.1006, 40.0931]]]],
       device='cuda:0', grad_fn=<AddBackward0>)
tensor([[[[43.0590, 66.0652, 87.5816,  ..., 38.2939, 56.6799, 39.9333]]],


        [[[41.8966, 64.9154, 84.5414,  ..., 38.8669, 59.0957, 40.0924]]],


        [[[41.8975, 64.9165, 84.5440,  ..., 38.8665, 59.0936, 40.0922]]],


        ...,


        [[[43.7998, 65.1398, 82.3979,  ..., 37.7072, 52.0964, 34.9449]]],


        [[[41.8965, 64.9153, 84.5410,  ..., 38.8669, 59.0960, 40.0924]]],


        [[[41.8961, 64.9152, 84.5404,  ..., 38.8672, 59.0965, 40.0924]]]],
       devic

  0%|          | 0/2 [00:00<?, ?it/s]

tensor([[[[40.3318, 67.5361, 91.3445,  ..., 35.3144, 54.8196, 46.2252]]],


        [[[42.4870, 63.6081, 85.8827,  ..., 39.6528, 60.2380, 41.4005]]],


        [[[40.3314, 67.5366, 91.3447,  ..., 35.3147, 54.8192, 46.2249]]],


        ...,


        [[[40.3296, 67.5388, 91.3454,  ..., 35.3160, 54.8174, 46.2239]]],


        [[[42.4870, 63.6073, 85.8817,  ..., 39.6527, 60.2389, 41.4007]]],


        [[[43.8400, 65.3707, 89.8110,  ..., 39.1165, 57.1245, 41.0329]]]],
       device='cuda:0', grad_fn=<AddBackward0>)
tensor([[[[44.5397, 64.2161, 84.0991,  ..., 38.5888, 52.0682, 36.5485]]],


        [[[44.5402, 64.2155, 84.0990,  ..., 38.5885, 52.0685, 36.5487]]],


        [[[44.5419, 64.2136, 84.0984,  ..., 38.5874, 52.0699, 36.5496]]],


        ...,


        [[[42.3787, 63.1369, 85.2073,  ..., 39.6303, 59.8421, 41.8980]]],


        [[[42.3778, 63.1332, 85.2015,  ..., 39.6304, 59.8474, 41.8991]]],


        [[[42.3767, 63.1298, 85.1958,  ..., 39.6306, 59.8523, 41.9001]]]],
       devic

100%|██████████| 2/2 [00:00<00:00, 16.27it/s]


In [28]:
model = GCN()
model_save_path = os.path.join('st_DynGNN_best.pt')
state_dict = {k.replace('module.', ''): v for k, v in torch.load(model_save_path).items()}
model.load_state_dict(state_dict)
model.eval()
h_a, h_f = model(dgl_graph, x)

h_a = torch.squeeze(h_a)
h_a.shape

DGLError: Invalid key "0". Must be one of the edge types.

In [32]:
h_f

tensor([[[[ 0.0000, 46.5279,  0.0000,  ..., 32.2680,  0.0000,  0.0000]]],


        [[[ 0.0000, 46.5266,  0.0000,  ..., 32.2655,  0.0000,  0.0000]]],


        [[[ 0.0000, 46.5265,  0.0000,  ..., 32.2652,  0.0000,  0.0000]]],


        ...,


        [[[ 0.0000, 43.4879,  0.0000,  ..., 31.6806,  0.0000,  0.0000]]],


        [[[ 0.0000, 43.4868,  0.0000,  ..., 31.6780,  0.0000,  0.0000]]],


        [[[ 0.0000, 43.4882,  0.0000,  ..., 31.6811,  0.0000,  0.0000]]]],
       grad_fn=<ReluBackward0>)

In [95]:
m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T


K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
    for j in range(120):
        if int(j/K) == i:
            d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
    for j in range(10):
        W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W

criterion = torch.nn.MSELoss()
loss = criterion(torch.squeeze(h_a)+torch.squeeze(h_f), x)  + \
                10*torch.std(torch.diff(h_a)) +\
                0 * torch.var(h_a) +\
                100 * torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(torch.squeeze(h_f), 0, 1))[1])) +\
                5 * torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(torch.squeeze(h_f), E.T).float(), L.float()), torch.matmul(E, torch.squeeze(h_f).T).float()), 0))
loss

tensor(1478.1057, grad_fn=<AddBackward0>)

In [17]:
import torch
import psutil

# if torch.cuda.is_available():
#     num_gpus = torch.cuda.device_count()
#     print("Number of available GPUs:", num_gpus)
# else:
num_cpus = psutil.cpu_count(logical=False)  # Count physical CPUs (sockets)
print("Number of available physical CPUs:", num_cpus)


Number of available physical CPUs: 128


In [33]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GATConv, GATv2Conv, GCNConv, TransformerConv, AGNNConv, FusedGATConv, ChebConv, SAGEConv, GraphConv, TAGConv, GMMConv 
from scipy.stats import linregress

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(42)



        self.conv1 = GCNConv(120, 60)
        self.conv2 = GCNConv(60, 120)
        self.conv3 = GCNConv(120, 60)
        self.conv4 = GCNConv(60, 120)
        
    

    def forward(self, x, edge_index):

  
        h_1 = self.conv1(x, edge_index)
        h_a = self.conv2(h_1, edge_index)


        h_2 = self.conv3(x, edge_index)
        h_f = self.conv4(h_2, edge_index)

        return h_a, h_f
model = GCN()
print(model)

GCN(
  (conv1): GCNConv(120, 60)
  (conv2): GCNConv(60, 120)
  (conv3): GCNConv(120, 60)
  (conv4): GCNConv(60, 120)
)


In [34]:
from sklearn.metrics import mean_squared_error
import torch
import torch.nn.functional as F

# Lambda1 = 0.1
# Lambda2 = 250
# Lambda3 = 1

Lambda1 = 0
Lambda2 = 100
Lambda3 = 5

m1 = [[1] for i in range(1,121)]
m2 = [[i] for i in range(1,121)]
m = torch.Tensor([m1, m2])
m = torch.squeeze(m).T

K = 12
d = [ [ 0 for y in range(120) ] for x in range(10) ]
for i in range(10):
  for j in range(120):
    if int(j/K) == i:
      d[i][j] = 1/K
E = torch.as_tensor(d)

W = [ [ 1 for y in range(10) ] for x in range(10) ]
for i in range(10):
  for j in range(10):
    W[i][j] = abs(j-i)
W = torch.as_tensor(W)
W = W.fill_diagonal_(1)


I = [ 1 for x in range(10) ]
I = torch.as_tensor(I)
L = torch.diag(torch.matmul(W, I), 0) - W

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay= 0.02)


def train(x, edge_index, Lambda1, Lambda2):
  optimizer.zero_grad()
  h_a, h_f = model(x, edge_index)
  loss = criterion(h_a+h_f, x)
  # loss = criterion(h_a+h_f, x)  + \
  # 10*torch.std(torch.diff(h_a)) +\
  # Lambda1 * torch.var(h_a) +\
  # Lambda2 * torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(h_f, 0, 1))[1])) +\
  # Lambda3 * torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(h_f, E.T).long(), L.long()), torch.matmul(E, h_f.T).long()), 0))

  # print(criterion(h_a + h_f, x))
  # print(torch.sum(torch.abs(torch.matmul(torch.matmul(torch.linalg.inv(torch.matmul(torch.transpose(m, 0, 1), m)), torch.transpose(m, 0, 1)), torch.transpose(h_f, 0, 1))[1])))
  # print(torch.sum(torch.diagonal(torch.matmul(torch.matmul(torch.matmul(h_f, E.T).long(), L.long()), torch.matmul(E, h_f.T).long()), 0)))

  loss.backward()
  optimizer.step()
  return loss, criterion(h_a+h_f, x), h_a, h_f

epochs = range(1, 100)
losses = []
embeddings = []
min_loss = 1000
for epoch in epochs:
  loss, loss_2, h_a, h_f= train(x, edge_index, Lambda1, Lambda2)
  losses.append(loss)
  print(f"Epoch: {epoch}\tLoss: {loss:.4f}")
  # if loss_2 < 100:
  #   break


Epoch: 1	Loss: 4043.5540
Epoch: 2	Loss: 3077.6011
Epoch: 3	Loss: 1203.8195
Epoch: 4	Loss: 1130.1881
Epoch: 5	Loss: 788.4680
Epoch: 6	Loss: 574.6251
Epoch: 7	Loss: 534.2384
Epoch: 8	Loss: 519.5971
Epoch: 9	Loss: 398.7972
Epoch: 10	Loss: 292.3871
Epoch: 11	Loss: 272.8162
Epoch: 12	Loss: 253.1906
Epoch: 13	Loss: 203.6208
Epoch: 14	Loss: 169.9689
Epoch: 15	Loss: 150.9902
Epoch: 16	Loss: 138.5900
Epoch: 17	Loss: 126.7226
Epoch: 18	Loss: 110.5001
Epoch: 19	Loss: 104.1171
Epoch: 20	Loss: 88.5925
Epoch: 21	Loss: 69.0048
Epoch: 22	Loss: 61.3695
Epoch: 23	Loss: 55.2481
Epoch: 24	Loss: 49.1813
Epoch: 25	Loss: 43.2701
Epoch: 26	Loss: 38.8014
Epoch: 27	Loss: 36.3555
Epoch: 28	Loss: 34.8477
Epoch: 29	Loss: 32.6112
Epoch: 30	Loss: 31.5829
Epoch: 31	Loss: 29.7149
Epoch: 32	Loss: 28.3097
Epoch: 33	Loss: 26.9053
Epoch: 34	Loss: 23.6492
Epoch: 35	Loss: 21.8967
Epoch: 36	Loss: 21.4657
Epoch: 37	Loss: 21.1018
Epoch: 38	Loss: 20.0730
Epoch: 39	Loss: 19.3213


Epoch: 40	Loss: 19.2897
Epoch: 41	Loss: 18.5695
Epoch: 42	Loss: 17.1709
Epoch: 43	Loss: 16.2519
Epoch: 44	Loss: 15.7972
Epoch: 45	Loss: 15.4778
Epoch: 46	Loss: 15.6919
Epoch: 47	Loss: 16.8263
Epoch: 48	Loss: 20.7117
Epoch: 49	Loss: 32.2773
Epoch: 50	Loss: 64.5979
Epoch: 51	Loss: 132.1909
Epoch: 52	Loss: 203.4060
Epoch: 53	Loss: 146.8800
Epoch: 54	Loss: 21.9300
Epoch: 55	Loss: 51.9439
Epoch: 56	Loss: 116.7872
Epoch: 57	Loss: 40.7097
Epoch: 58	Loss: 23.9973
Epoch: 59	Loss: 80.8747
Epoch: 60	Loss: 29.1475
Epoch: 61	Loss: 23.9380
Epoch: 62	Loss: 58.4518
Epoch: 63	Loss: 14.3489
Epoch: 64	Loss: 30.3226
Epoch: 65	Loss: 36.9141
Epoch: 66	Loss: 9.1399
Epoch: 67	Loss: 33.3893
Epoch: 68	Loss: 17.2631
Epoch: 69	Loss: 15.0563
Epoch: 70	Loss: 26.4170
Epoch: 71	Loss: 8.4907
Epoch: 72	Loss: 21.2588
Epoch: 73	Loss: 14.3692
Epoch: 74	Loss: 11.1695
Epoch: 75	Loss: 18.6672
Epoch: 76	Loss: 8.0061
Epoch: 77	Loss: 15.1045
Epoch: 78	Loss: 11.3733
Epoch: 79	Loss: 9.2950
Epoch: 80	Loss: 13.6646
Epoch: 81	Loss: 

In [36]:
h_f

tensor([[22.3434, 26.4559, 25.8054,  ..., 20.5591, 42.3927, 17.1584],
        [22.3434, 26.4559, 25.8054,  ..., 20.5591, 42.3927, 17.1584],
        [22.3434, 26.4559, 25.8054,  ..., 20.5591, 42.3927, 17.1584],
        ...,
        [22.9697, 25.4002, 23.6304,  ..., 17.8749, 37.6075, 12.5646],
        [22.9697, 25.4002, 23.6304,  ..., 17.8749, 37.6075, 12.5646],
        [22.9697, 25.4002, 23.6304,  ..., 17.8749, 37.6075, 12.5646]],
       grad_fn=<AddBackward0>)